Notebook to conduct a full analyis of burned area, climate and fire weather indices on an individual ecoregion level.

Includes the bias correction process for each climate/fire weather variable. Also includes code to set Winter months to 0 (where necessary) if erroneous values exist.

Edit file paths as necessary

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.dates as mdates
import seaborn as sns

In [ ]:
# Load files
# Specify region and root folder
region = 'nortsib'
region_model = 'nesibco'
root = f'/home/users/clelland/Model/Analysis/CMIP and FWI time series/Ecoregion CSVs/{region}'

# Import processed model data
df_model = pd.read_csv(f'/home/users/clelland/Model/Analysis/Ecoregion plots combined/area_timeseries_{region_model}_all.csv', parse_dates=['time'], index_col='time')
df_actual = pd.read_csv('/home/users/clelland/Model/Analysis/Fire actual 2001-2024.csv', parse_dates=['date'], index_col='date')[f'{region}']

# Import CSV files from folder
df_e5l = pd.read_csv(f'{root}/e5l_2001_2023_{region}.csv', parse_dates=['date'], index_col='date')
df_access_hist_climate = pd.read_csv(f'{root}/access_climate_2001_2014_{region}.csv', parse_dates=['date'], index_col='date')
df_access_126_climate = pd.read_csv(f'{root}/access_ssp126_climate_2015_2100_{region}.csv', parse_dates=['date'], index_col='date')
df_access_245_climate = pd.read_csv(f'{root}/access_ssp245_climate_2015_2100_{region}.csv', parse_dates=['date'], index_col='date')
df_access_370_climate = pd.read_csv(f'{root}/access_ssp370_climate_2015_2100_{region}.csv', parse_dates=['date'], index_col='date')
df_mri_hist_climate = pd.read_csv(f'{root}/mri_climate_2001_2014_{region}.csv', parse_dates=['date'], index_col='date')
df_mri_126_climate = pd.read_csv(f'{root}/mri_ssp126_climate_2015_2100_{region}.csv', parse_dates=['date'], index_col='date')
df_mri_245_climate = pd.read_csv(f'{root}/mri_ssp245_climate_2015_2100_{region}.csv', parse_dates=['date'], index_col='date')
df_mri_370_climate = pd.read_csv(f'{root}/mri_ssp370_climate_2015_2100_{region}.csv', parse_dates=['date'], index_col='date')
df_cems = pd.read_csv(f'{root}/cems_2001_2023_{region}.csv', parse_dates=['date'], index_col='date')
df_access_hist_fwi = pd.read_csv(f'{root}/access_fwi_2001_2014_{region}.csv', parse_dates=['date'], index_col='date')
df_access_126_fwi = pd.read_csv(f'{root}/access_ssp126_fwi_2015_2100_{region}.csv', parse_dates=['date'], index_col='date')
df_access_245_fwi = pd.read_csv(f'{root}/access_ssp245_fwi_2015_2100_{region}.csv', parse_dates=['date'], index_col='date')
df_access_370_fwi = pd.read_csv(f'{root}/access_ssp370_fwi_2015_2100_{region}.csv', parse_dates=['date'], index_col='date')
df_mri_hist_fwi = pd.read_csv(f'{root}/mri_fwi_2001_2014_{region}.csv', parse_dates=['date'], index_col='date')
df_mri_126_fwi = pd.read_csv(f'{root}/mri_ssp126_fwi_2015_2100_{region}.csv', parse_dates=['date'], index_col='date')
df_mri_245_fwi = pd.read_csv(f'{root}/mri_ssp245_fwi_2015_2100_{region}.csv', parse_dates=['date'], index_col='date')
df_mri_370_fwi = pd.read_csv(f'{root}/mri_ssp370_fwi_2015_2100_{region}.csv', parse_dates=['date'], index_col='date')

# Analysis

## BA

In [ ]:
# ba
df_ba = pd.DataFrame({
    'Actual': df_actual,
    'ACCESS_SSP126': df_model['access 126'],
    'ACCESS_SSP245': df_model['access 245'],
    'ACCESS_SSP370': df_model['access 370'],
    'MRI_SSP126': df_model['mri 126'],
    'MRI_SSP245': df_model['mri 245'],
    'MRI_SSP370': df_model['mri 370']
})

df_ba.sort_index(inplace=True)

# Divide columns by a scaling factor
df_ba.iloc[:, 1:4] = df_ba.iloc[:, 1:4] / 16 # ACCESS
df_ba.iloc[:, 4:7] = df_ba.iloc[:, 4:7] / 8 # MRI
df_ba

### BA Plot

In [ ]:
df_ba_plot = df_ba.copy()
df_ba_plot.index = pd.to_datetime(df_ba_plot.index)
df_ba_plot = df_ba_plot.resample('YE').sum()
df_ba_plot.index = df_ba_plot.index + pd.offsets.Day(1)
df_ba_plot.index = df_ba_plot.index - pd.offsets.YearBegin(1)
df_ba_plot = df_ba_plot.dropna(axis=1, how='all')

# Set figure size
plt.figure(figsize=(14, 6))

# Plot first column (only up to 2024)
first_col = df_ba_plot.columns[0]
df_first = df_ba_plot.loc[:'2024-12-31', first_col]
plt.plot(df_first.index, df_first, label=first_col, linewidth=2)

# Add extreme year dashed line
mean_actual = df_first.mean()
std_actual = df_first.std()
threshold = mean_actual + 2 * std_actual
plt.axhline(threshold, color='gray', linestyle='--', linewidth=1.5)

# Plot all other columns (from 2025 onwards)
for column in df_ba_plot.columns[1:]:
    df_other = df_ba_plot.loc['2025-01-01':, column]
    plt.plot(df_other.index, df_other, label=column)

# Set major ticks to every 5 years
years = pd.date_range(start='2000', end='2100', freq='5YS')
plt.gca().set_xticks(years)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Add title and labels
plt.title(f'BA Time Series Yearly sum for {region}')
plt.xlabel('Year')
plt.ylabel('Mha')
plt.legend()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
           ncol=3, fancybox=True, shadow=True) # Legend below
plt.grid(True)
plt.tight_layout()
#plt.savefig(f'/home/users/clelland/Model/Analysis/CMIP and FWI time series/Ecoregion CSVs/{region}/ba_plot_{region}_2001_2100.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
print(df_ba_plot.loc[:'2024-01-01']['Actual'].mean())
print(df_ba_plot.loc['2025-01-01':]['Model ACCESS_SSP126'].mean())
print(df_ba_plot.loc['2025-01-01':]['Model ACCESS_SSP245'].mean())
print(df_ba_plot.loc['2025-01-01':]['Model ACCESS_SSP370'].mean())
print(df_ba_plot.loc['2025-01-01':]['Model MRI_SSP126'].mean())
print(df_ba_plot.loc['2025-01-01':]['Model MRI_SSP245'].mean())
print(df_ba_plot.loc['2025-01-01':]['Model MRI_SSP370'].mean())

### Actual seasonal BA

In [ ]:
# TOTAL BA PER DECADE - ACTUAL ONLY
# Step 1: Add year, month, and decade columns
df_season = df_ba.copy().reset_index().rename(columns={'index':'time'})
df_season['time'] = pd.to_datetime(df_season['time'])
df_season.set_index('time', inplace=True)
df_season = df_season.loc[:'2024-12-31']

df_season['year'] = df_season.index.year
df_season['month'] = df_season.index.month
df_season['decade'] = ((df_season['year'] - 1) // 10) * 10 + 1

burn_column = 'Actual'

# Step 3: Group by decade and month, summing the burn values
monthly_burn = df_season.groupby(['decade', 'month'])[burn_column].sum()

# Step 4: Calculate yearly totals for each decade
yearly_totals = df_season.groupby('decade')[burn_column].sum()

# Step 5: Calculate percentage of yearly burn for each month in each decade
percent_burn = monthly_burn.div(yearly_totals, level='decade') * 100

# Step 6: Plot the results
plt.figure(figsize=(10, 6))
for decade in percent_burn.index.levels[0]:  # Loop over each decade
    label = f"{decade - 1}s"
    plt.plot(percent_burn.loc[decade].index, percent_burn.loc[decade].values, label=label) # e.g., 2021 → "2020s"

plt.xticks(ticks=range(1, 13), labels=[
    'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
    'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'
])

plt.xlabel('Month')
plt.ylabel('Percent of Yearly Burn (%)')
plt.title(f'Actual Total Monthly Burn Percentages by Decade for {region}')
plt.grid(True)
plt.legend(title='Decade')
plt.tight_layout()
#plt.savefig(f'/home/users/clelland/Model/Analysis/CMIP and FWI time series/Ecoregion CSVs/{region}/ba_seasonality_plot_{region}_2001_2023.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
# MEAN BA AND SD PER DECADE - ACTUAL ONLY
# Step 1: Add year, month, and decade columns (already done)
df_season = df_ba.copy().reset_index().rename(columns={'index':'time'})
df_season['time'] = pd.to_datetime(df_season['time'])
df_season.set_index('time', inplace=True)
df_season = df_season.loc[:'2024-12-31']

df_season['year'] = df_season.index.year
df_season['month'] = df_season.index.month
df_season['decade'] = ((df_season['year'] - 1) // 10) * 10 + 1

burn_column = 'Actual'

# Step 2: Group by decade, year, and month to compute actual monthly burn
monthly_grouped = df_season.groupby(['decade', 'year', 'month'])[burn_column].sum()

# Step 3: Compute yearly totals per decade and year
yearly_grouped = df_season.groupby(['decade', 'year'])[burn_column].sum()

# Step 4: Calculate percent of yearly burn per month per year
# We need to divide monthly values by the corresponding yearly total
monthly_percent = monthly_grouped / yearly_grouped.loc[monthly_grouped.index.droplevel('month')] * 100

# Step 5: Calculate mean and std dev across years for each decade and month
mean_percent = monthly_percent.groupby(['decade', 'month']).mean()
std_percent = monthly_percent.groupby(['decade', 'month']).std()

# Step 6: Plot with error bars
plt.figure(figsize=(10, 6))
for decade in mean_percent.index.levels[0]:
    label = f"{decade - 1}s"
    months = mean_percent.loc[decade].index
    values = mean_percent.loc[decade].values
    errors = std_percent.loc[decade].values

    # Calculate upper and lower bounds, ensuring lower bound is not negative
    lower = np.maximum(values - errors, 0)
    upper = np.minimum(values + errors, 100)

    # Plot mean line
    plt.plot(months, values, label=label)

    # Plot shaded area for ±1 standard deviation
    plt.fill_between(
        months,
        lower,
        upper,
        alpha=0.2
    )

plt.xticks(ticks=range(1, 13), labels=[
    'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
    'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'
])

plt.xlabel('Month')
plt.ylabel('Percent of Yearly Burn (%)')
plt.title(f'Actual Mean Monthly Burn Percentages by Decade for {region}\n(with standard deviation)')
plt.grid(True)
plt.legend(title='Decade')
plt.tight_layout()
#plt.savefig(f'/home/users/clelland/Model/Analysis/CMIP and FWI time series/Ecoregion CSVs/{region}/ba_seasonality_plot_with_sd_{region}_2001_2023.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
# ALL INDIVIDUAL YEARS PER SCENARIO - ACTUAL
# Step 1: Add year, month, and decade columns
df_season = df_ba.copy().reset_index().rename(columns={'index':'time'})
df_season['time'] = pd.to_datetime(df_season['time'])
df_season.set_index('time', inplace=True)
df_season = df_season.loc[:'2024-12-31']

# Step 1: Add year, month columns
df_season['year'] = df_season.index.year
df_season['month'] = df_season.index.month

burn_column = 'Actual'

# Step 3: Group by year and month, summing the burn values
monthly_burn = df_season.groupby(['year', 'month'])[burn_column].sum()

# Step 4: Calculate yearly totals for each year
yearly_totals = df_season.groupby('year')[burn_column].sum()

# Step 5: Calculate percentage of yearly burn for each month in each year
percent_burn = monthly_burn.div(yearly_totals, level='year') * 100

# Step 6: Set up colormap and get a color gradient
cmap = cm.viridis
norm = plt.Normalize(vmin=percent_burn.index.get_level_values('year').min(),
                     vmax=percent_burn.index.get_level_values('year').max())
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])

# Step 7: Plot each year with a different shade
fig, ax = plt.subplots(figsize=(10, 6))
for year in percent_burn.index.levels[0]:  # Loop over each year
    color = cmap(norm(year))  # Get color for the year from the colormap
    ax.plot(percent_burn.loc[year].index, percent_burn.loc[year].values, color=color)

# Step 8: Add colorbar to the right of the plot
cbar = fig.colorbar(sm, ax=ax, pad=0.01)
cbar.set_label('Year')

# Step 9: Set labels and title
ax.set_xticks(range(1, 13))
ax.set_xticklabels([
    'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
    'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'
])
ax.set_xlabel('Month')
ax.set_ylabel('Percent of Yearly Burn (%)')
ax.set_title(f'Actual Monthly Burn Percentages by Year for {region}')
ax.grid(True)

# Step 10: Show the plot
plt.tight_layout()
#plt.savefig(f'/home/users/clelland/Model/Analysis/CMIP and FWI time series/Ecoregion CSVs/{region}/ba_seasonality_plot_all_years_{region}_2001_2023.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

### Model seasonal BA

In [ ]:
# TOTAL BA PER DECADE - MODEL ONLY
# Step 1: Add year, month, and decade columns
df_season = df_ba.copy().reset_index().rename(columns={'index':'time'})
df_season['time'] = pd.to_datetime(df_season['time'])
df_season.set_index('time', inplace=True)
df_season = df_season.loc['2025-01-01':]

df_season['year'] = df_season.index.year
df_season['month'] = df_season.index.month
df_season['decade'] = ((df_season['year'] - 1) // 10) * 10 + 1

# Step 2: Choose the column of interest
burn_column = 'MRI_SSP370'

# Step 3: Group by decade and month, summing the burn values
monthly_burn = df_season.groupby(['decade', 'month'])[burn_column].sum()

# Step 4: Calculate yearly totals for each decade
yearly_totals = df_season.groupby('decade')[burn_column].sum()

# Step 5: Calculate percentage of yearly burn for each month in each decade
percent_burn = monthly_burn.div(yearly_totals, level='decade') * 100

# Step 6: Plot the results
plt.figure(figsize=(10, 6))
for decade in percent_burn.index.levels[0]:  # Loop over each decade
    label = f"{decade - 1}s"
    plt.plot(percent_burn.loc[decade].index, percent_burn.loc[decade].values, label=label) # e.g., 2021 → "2020s"

plt.xticks(ticks=range(1, 13), labels=[
    'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
    'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'
])

plt.xlabel('Month')
plt.ylabel('Percent of Yearly Burn (%)')
plt.title(f'Monthly Total Burn Percentages by Decade for {region} {burn_column}')
plt.grid(True)
plt.legend(title='Decade')
plt.tight_layout()
#plt.savefig(f'/home/users/clelland/Model/Analysis/CMIP and FWI time series/Ecoregion CSVs/{region}/ba_seasonality_plot_{region}_mri_370_2025_2100.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
# MEAN BA AND SD PER DECADE - MODEL ONLY
# Step 1: Add year, month, and decade columns (already done)
df_season = df_ba.copy().reset_index().rename(columns={'index':'time'})
df_season['time'] = pd.to_datetime(df_season['time'])
df_season.set_index('time', inplace=True)
df_season = df_season.loc['2025-01-01':]

df_season['year'] = df_season.index.year
df_season['month'] = df_season.index.month
df_season['decade'] = ((df_season['year'] - 1) // 10) * 10 + 1

# Step 2: Choose the column of interest
burn_column = 'ACCESS_SSP126'

# Step 2: Group by decade, year, and month to compute actual monthly burn
monthly_grouped = df_season.groupby(['decade', 'year', 'month'])[burn_column].sum()

# Step 3: Compute yearly totals per decade and year
yearly_grouped = df_season.groupby(['decade', 'year'])[burn_column].sum()

# Step 4: Calculate percent of yearly burn per month per year
# We need to divide monthly values by the corresponding yearly total
monthly_percent = monthly_grouped / yearly_grouped.loc[monthly_grouped.index.droplevel('month')] * 100

# Step 5: Calculate mean and std dev across years for each decade and month
mean_percent = monthly_percent.groupby(['decade', 'month']).mean()
std_percent = monthly_percent.groupby(['decade', 'month']).std()

# Step 6: Plot with error bars
plt.figure(figsize=(10, 6))
for decade in mean_percent.index.levels[0]:
    label = f"{decade - 1}s"
    months = mean_percent.loc[decade].index
    values = mean_percent.loc[decade].values
    errors = std_percent.loc[decade].values

    # Calculate upper and lower bounds, ensuring lower bound is not negative
    lower = np.maximum(values - errors, 0)
    upper = np.minimum(values + errors, 100)

    # Plot mean line
    plt.plot(months, values, label=label)

    # Plot shaded area for ±1 standard deviation
    plt.fill_between(
        months,
        lower,
        upper,
        alpha=0.2
    )
    
plt.xticks(ticks=range(1, 13), labels=[
    'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
    'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'
])

plt.xlabel('Month')
plt.ylabel('Percent of Yearly Burn (%)')
plt.title(f'Monthly Mean Burn Percentages by Decade for {region} {burn_column} \n(with standard deviation)')
plt.grid(True)
plt.legend(title='Decade')
plt.tight_layout()
#plt.savefig(f'/home/users/clelland/Model/Analysis/CMIP and FWI time series/Ecoregion CSVs/{region}/ba_seasonality_plot_with_sd_{region}_2001_2023.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
# Step 1: Calculate monthly percent per year (revised method from before)
df_season = df_ba.copy().reset_index().rename(columns={'index': 'time'})
df_season['time'] = pd.to_datetime(df_season['time'])
df_season.set_index('time', inplace=True)
df_season = df_season.loc['2025-01-01':]

df_season['year'] = df_season.index.year
df_season['month'] = df_season.index.month

burn_column = 'ACCESS_SSP126'

# Total burn per year
yearly_totals = df_season.groupby(['year'])[burn_column].sum()

# Monthly burn
monthly_burn = df_season.groupby(['year', 'month'])[burn_column].sum()

# Convert to DataFrame for easier handling
monthly_burn_df = monthly_burn.reset_index()
monthly_burn_df['yearly_total'] = monthly_burn_df['year'].map(yearly_totals)

# Compute percent
monthly_burn_df['percent'] = monthly_burn_df[burn_column] / monthly_burn_df['yearly_total'] * 100

# Add date for plotting
monthly_burn_df['date'] = pd.to_datetime(dict(year=monthly_burn_df['year'],
                                              month=monthly_burn_df['month'],
                                              day=15))
monthly_burn_df.set_index('date', inplace=True)

# Pivot the data to have years as columns, months as rows
heatmap_data = monthly_burn_df.pivot(index='month', columns='year', values='percent')

# Plot heatmap
plt.figure(figsize=(16, 6))
sns.heatmap(heatmap_data, cmap='YlOrRd', linewidths=0.5, linecolor='grey', annot=False, fmt=".1f", cbar_kws={'label': '% of Yearly Burn'})

# Beautify axis labels
plt.yticks(ticks=np.arange(12) + 0.5, labels=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                                              'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], rotation=0)
plt.xlabel('Year')
plt.ylabel('Month')
plt.title(f'Monthly Contribution to Yearly Burn (2025–2100) for {region} {burn_column}')
plt.tight_layout()
plt.show()

In [ ]:
# MEAN BA AND SD PER DECADE - MODEL ONLY
# Step 1: Add year, month, and decade columns (already done)
df_season = df_ba.copy().reset_index().rename(columns={'index':'time'})
df_season['time'] = pd.to_datetime(df_season['time'])
df_season.set_index('time', inplace=True)
df_season = df_season.loc['2025-01-01':]

df_season['year'] = df_season.index.year
df_season['month'] = df_season.index.month
df_season['decade'] = ((df_season['year'] - 1) // 10) * 10 + 1

# Step 2: Choose the column of interest
burn_column = 'ACCESS_SSP126'

# Step 2: Group by decade, year, and month to compute actual monthly burn
monthly_grouped = df_season.groupby(['decade', 'year', 'month'])[burn_column].sum()

# Step 3: Compute yearly totals per decade and year
yearly_grouped = df_season.groupby(['decade', 'year'])[burn_column].sum()

# Step 4: Calculate percent of yearly burn per month per year
# We need to divide monthly values by the corresponding yearly total
monthly_percent = monthly_grouped / yearly_grouped.loc[monthly_grouped.index.droplevel('month')] * 100

# Step 5: Calculate mean and std dev across years for each decade and month
mean_percent = monthly_percent.groupby(['decade', 'month']).mean()
std_percent = monthly_percent.groupby(['decade', 'month']).std()

# Reset and merge mean and std dev
mean_df = mean_percent.reset_index()
std_df = std_percent.reset_index()
combined_df = pd.merge(mean_df, std_df, on=['decade', 'month'], suffixes=('_mean', '_std'))

# Create pivot tables
mean_pivot = combined_df.pivot(index='month', columns='decade', values=f'{burn_column}_mean')
std_pivot = combined_df.pivot(index='month', columns='decade', values=f'{burn_column}_std')

# Format the decade labels (e.g., 2001 -> "2000s")
formatted_columns = [f"{int(decade) - 1}s" for decade in mean_pivot.columns]
mean_pivot.columns = formatted_columns
std_pivot.columns = formatted_columns

# Create annotation strings: "mean ± std"
annot = mean_pivot.round(1).astype(str) + " ± " + std_pivot.round(1).astype(str)

# Plot the heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(mean_pivot, cmap='YlOrRd', annot=annot, fmt='', linewidths=0.5, linecolor='grey',
            cbar_kws={'label': 'Mean % of Yearly Burn'})

# Y-axis: month names
plt.yticks(ticks=np.arange(12) + 0.5, labels=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                                              'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], rotation=0)

plt.xlabel('Decade')
plt.ylabel('Month')
plt.title(f'Monthly Mean ± Std Dev Burn (% of Yearly) by Decade for {region} {burn_column}')
plt.tight_layout()
plt.show()

In [ ]:
# ALL INDIVIDUAL YEARS PER SCENARIO
# Step 1: Add year, month, and decade columns
df_season = df_ba.copy().reset_index().rename(columns={'index':'time'})
df_season['time'] = pd.to_datetime(df_season['time'])
df_season.set_index('time', inplace=True)
df_season = df_season.loc['2025-01-01':]

# Step 1: Add year, month columns
df_season['year'] = df_season.index.year
df_season['month'] = df_season.index.month

# Step 2: Choose the column of interest
burn_column = 'ACCESS_SSP126'

# Step 3: Group by year and month, summing the burn values
monthly_burn = df_season.groupby(['year', 'month'])[burn_column].sum()

# Step 4: Calculate yearly totals for each year
yearly_totals = df_season.groupby('year')[burn_column].sum()

# Step 5: Calculate percentage of yearly burn for each month in each year
percent_burn = monthly_burn.div(yearly_totals, level='year') * 100

# Step 6: Set up colormap and get a color gradient
cmap = cm.viridis
norm = plt.Normalize(vmin=percent_burn.index.get_level_values('year').min(),
                     vmax=percent_burn.index.get_level_values('year').max())
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])

# Step 7: Plot each year with a different shade
fig, ax = plt.subplots(figsize=(10, 6))
for year in percent_burn.index.levels[0]:  # Loop over each year
    color = cmap(norm(year))  # Get color for the year from the colormap
    ax.plot(percent_burn.loc[year].index, percent_burn.loc[year].values, color=color)

# Step 8: Add colorbar to the right of the plot
cbar = fig.colorbar(sm, ax=ax, pad=0.01)
cbar.set_label('Year')

# Step 9: Set labels and title
ax.set_xticks(range(1, 13))
ax.set_xticklabels([
    'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
    'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'
])
ax.set_xlabel('Month')
ax.set_ylabel('Percent of Yearly Burn (%)')
ax.set_title(f'Monthly Burn Percentages by Year for {region} {burn_column}')
ax.grid(True)

# Step 10: Show the plot
plt.tight_layout()
#plt.savefig(f'/home/users/clelland/Model/Analysis/CMIP and FWI time series/Ecoregion CSVs/{region}/ba_seasonality_plot_all_years_{region}_mri_370_2025_2100.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
# Ensure datetime index
df_ba_plot = df_ba.copy()
df_ba_plot.index = pd.to_datetime(df_ba_plot.index)
df_ba_plot = df_ba_plot.dropna(axis=1, how='all')

# Determine year range
all_years = df_ba_plot.index.year.unique()
min_year, max_year = all_years.min(), all_years.max()
year_range = pd.Index(range(min_year, max_year + 1))

# Build DataFrame: years as index, models as columns, values = month of max burn
max_month_df = pd.DataFrame(index=year_range)

for column in df_ba_plot.columns:
    monthly_data = df_ba_plot[column].groupby(df_ba_plot.index.year)

    def get_max_month(group):
        if group.isna().all() or group.sum() == 0:
            return np.nan  # No valid data
        else:
            #return group.idxmax().month # Original
            max_val = group.max()
            max_months = group[group == max_val].index.month
            # Choose the month closest to June (month 6)
            return max_months[np.abs(max_months - 6).argmin()]

    max_months = monthly_data.apply(get_max_month)
    max_month_df[column] = max_months.reindex(year_range)

# Plot
plt.figure(figsize=(14, 6))

for column in max_month_df.columns:
    # Original (jittery) line
    plt.plot(
        max_month_df.index,
        max_month_df[column],
        label=column,
        linestyle='-',
        alpha=0.4  # lighter line
    )
    
    # Rolling 5-year mean
    rolling = max_month_df[column].rolling(window=5, min_periods=3).mean()
    plt.plot(
        rolling.index,
        rolling,
        linestyle='-',
        linewidth=2,
        alpha=0.9,
        label=f'{column}'
    )

# Y-axis: months, flipped
plt.yticks(ticks=range(1, 13), labels=[
    'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
    'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'
])
#plt.ylim(12.5, 0.5) # All months
plt.ylim(10.5, 2.5) # Mar to Oct

# X-axis: every 5 years
years = pd.date_range(start='2000', end='2100', freq='5YS').year
plt.xticks(ticks=years)

# Titles and layout
plt.title(f'5-year Rolling Mean of Month of Maximum Burned Area per Year – {region}')
plt.xlabel('Year')
plt.ylabel('Month of Max Burn')
plt.grid(True)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
           ncol=3, fancybox=True, shadow=True)
plt.tight_layout()

#plt.savefig(f'/home/users/clelland/Model/Analysis/Summary stats/BA/ba_max_month_{region}.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

### Processing - set erroneous months to 0 (if necessary)

In [ ]:
df_ba_edited = df_ba.copy()
model_columns = [col for col in df_ba_edited.columns if col != 'Actual']

# Create an empty DataFrame to store results
min_by_month = pd.DataFrame(index=range(1, 13), columns=model_columns)

# Fill the DataFrame with minimum values for each column and month
for month in range(1, 13):
    month_mask = df_ba_edited.index.month == month
    for col in model_columns:
        min_val = df_ba_edited.loc[month_mask, col].min()
        min_by_month.loc[month, col] = min_val

# Rename index for clarity
min_by_month.index.name = 'month'

min_by_month

In [ ]:
# Ensure min_by_month values are floats
min_by_month = min_by_month.astype(float)

# Apply subtraction of monthly minimums
for month in range(1, 13):
    # Boolean mask for rows in the current month
    month_mask = df_ba_edited.index.month == month
    # Subtract corresponding row from min_by_month (broadcast across rows)
    df_ba_edited.loc[month_mask, min_by_month.columns] = (
        df_ba_edited.loc[month_mask, min_by_month.columns] - min_by_month.loc[month]
    )

# Set any negative values to 0
df_ba_edited[df_ba_edited < 0] = 0
df_ba_edited

In [ ]:
# Plot edited df
df_ba_plot = df_ba_edited.copy()
df_ba_plot.index = pd.to_datetime(df_ba_plot.index)
df_ba_plot = df_ba_plot.resample('YE').sum()
df_ba_plot.index = df_ba_plot.index + pd.offsets.Day(1)
df_ba_plot.index = df_ba_plot.index - pd.offsets.YearBegin(1)
df_ba_plot = df_ba_plot.dropna(axis=1, how='all')

# Set figure size
plt.figure(figsize=(14, 6))

# Plot first column (only up to 2024)
first_col = df_ba_plot.columns[0]
df_first = df_ba_plot.loc[:'2024-12-31', first_col]
plt.plot(df_first.index, df_first, label=first_col, linewidth=2)

# Plot all other columns (from 2025 onwards)
for column in df_ba_plot.columns[1:]:
    df_other = df_ba_plot.loc['2025-01-01':, column]
    plt.plot(df_other.index, df_other, label=column)

# Set major ticks to every 5 years
years = pd.date_range(start='2000', end='2100', freq='5YS')
plt.gca().set_xticks(years)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Add title and labels
plt.title(f'Adjusted BA Time Series Yearly sum for {region}')
plt.xlabel('Year')
plt.ylabel('Mha')
plt.legend()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
           ncol=3, fancybox=True, shadow=True) # Legend below
plt.grid(True)
plt.tight_layout()
#plt.savefig(f'/home/users/clelland/Model/Analysis/CMIP and FWI time series/Ecoregion CSVs/{region}/ba_plot_{region}_2001_2100.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
# Save updated CSV
df_ba_save = df_ba_edited.copy()
df_ba_save.iloc[:, 1:] = df_ba_save.iloc[:, 1:] * 10 # Convert back to original raw values
df_ba_save.drop('Actual', axis=1, inplace=True)
df_ba_save.dropna(how='all', inplace=True)
df_ba_save = df_ba_save.reset_index().rename(columns={'index': 'time'})
new_column_names = {
    'ACCESS_SSP126': 'access 126',
    'ACCESS_SSP245': 'access 245',
    'ACCESS_SSP370': 'access 370',
    'MRI_SSP126': 'mri 126',
    'MRI_SSP245': 'mri 245',
    'MRI_SSP370': 'mri 370'
}
df_ba_save.rename(columns=new_column_names, inplace=True)
df_ba_save.to_csv(f'/home/users/clelland/Model/Analysis/Ecoregion plots combined/area_timeseries_{region_model}_all.csv', index=False)
df_ba_save

## Climate

### RH

#### Process and bias correction

In [ ]:
# rh
df_rh = pd.DataFrame({
    'ERA5-Land': df_e5l['rh'],
    'ACCESS_HIST': df_access_hist_climate['rh'],
    'ACCESS_SSP126': df_access_126_climate['rh'],
    'ACCESS_SSP245': df_access_245_climate['rh'],
    'ACCESS_SSP370': df_access_370_climate['rh'],
    'MRI_HIST': df_mri_hist_climate['rh'],
    'MRI_SSP126': df_mri_126_climate['rh'],
    'MRI_SSP245': df_mri_245_climate['rh'],
    'MRI_SSP370': df_mri_370_climate['rh'],
})

df_rh.sort_index(inplace=True)
df_rh

In [ ]:
# Bias correction
df_bias = df_rh.copy()

# First, ensure the index is datetime
df_bias.index = pd.to_datetime(df_bias.index)

# Add month as a new column
df_bias['month'] = df_bias.index.month

# Create a dictionary to store results
diff_results = {}

# 1. For 2001–2014 (historical)
hist_period = (df_bias.index >= '2001-01-01') & (df_bias.index <= '2014-12-31')

for model in ['ACCESS_HIST', 'MRI_HIST']:
    diff = df_bias.loc[hist_period, 'ERA5-Land'] - df_bias.loc[hist_period, model]
    monthly_diff = diff.groupby(df_bias.loc[hist_period, 'month']).mean()
    diff_results[model] = monthly_diff

# 2. For 2015–2023 (future scenarios)
future_period = (df_bias.index >= '2015-01-01') & (df_bias.index <= '2023-12-31')

for model in ['ACCESS_SSP126', 'ACCESS_SSP245', 'ACCESS_SSP370',
              'MRI_SSP126', 'MRI_SSP245', 'MRI_SSP370']:
    diff = df_bias.loc[future_period, 'ERA5-Land'] - df_bias.loc[future_period, model]
    monthly_diff = diff.groupby(df_bias.loc[future_period, 'month']).mean()
    diff_results[model] = monthly_diff

# 3. Combine all differences into a dataframe
diff_df = pd.DataFrame(diff_results)

# Optional: sort the dataframe by month
diff_df.index.name = 'month'
diff_df = diff_df.sort_index()

diff_df

In [ ]:
# 1. Define the period you want to correct
hist_correction_period = (df_bias.index >= '2001-01-01') & (df_bias.index <= '2014-12-31')
future_correction_period = (df_bias.index >= '2015-01-01') & (df_bias.index <= '2100-12-31')

# 2. Loop through each model/scenario you want to correct
corrected_data = {}
corrected_data['ERA5-Land'] = df_bias['ERA5-Land']

for model in ['ACCESS_HIST', 'MRI_HIST']:
    # Select the original model data for 2001–2014
    model_data = df_bias.loc[hist_correction_period, model].copy()
    
    # Get the corresponding month for each row
    months = df_bias.loc[hist_correction_period, 'month']
    
    # Add the bias correction based on the month
    corrected_values = model_data + months.map(diff_df[model])
    
    # Store in a dictionary
    corrected_data[model] = corrected_values

for model in ['ACCESS_SSP126', 'ACCESS_SSP245', 'ACCESS_SSP370',
              'MRI_SSP126', 'MRI_SSP245', 'MRI_SSP370']:
    # Select the original model data for 2015–2100
    model_data = df_bias.loc[future_correction_period, model].copy()
    
    # Get the corresponding month for each row
    months = df_bias.loc[future_correction_period, 'month']
    
    # Subtract the bias correction based on the month
    corrected_values = model_data + months.map(diff_df[model])
    
    # Store in a dictionary
    corrected_data[model] = corrected_values

# 3. Combine corrected data into a dataframe
corrected_df_rh = pd.DataFrame(corrected_data, index=df_bias.index)

corrected_df_rh

#### Plot

In [ ]:
df_rh_plot = df_rh.copy() # Original data
#df_rh_plot = corrected_df_rh.copy() # Corrected data
df_rh_plot.index = pd.to_datetime(df_rh_plot.index)
#df_rh_plot = df_rh_plot.loc['2001':'2014']
df_rh_plot = df_rh_plot.loc['2001':'2023']
df_rh_plot = df_rh_plot.dropna(axis=1, how='all')

# Set figure size
plt.figure(figsize=(14, 6))

# Plot each column
for column in df_rh_plot.columns:
    plt.plot(df_rh_plot.index, df_rh_plot[column], label=column)

# Set major ticks to every 5 years
#years = pd.date_range(start='2001', end='2015', freq='YS')
years = pd.date_range(start='2001', end='2023', freq='YS')
plt.gca().set_xticks(years)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Add title and labels
plt.title(f'RH Time Series for {region}')
#plt.title(f'Corrected RH Time Series for {region}')
plt.xlabel('Year')
plt.ylabel('%')
plt.legend()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
           ncol=3, fancybox=True, shadow=True) # Legend below
plt.grid(True)
plt.tight_layout()
#plt.savefig(f'/home/users/clelland/Model/Analysis/CMIP and FWI time series/Ecoregion CSVs/{region}/rh_plot_{region}_2001_2023.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
# Set figure size
plt.figure(figsize=(14, 6))

#df_rh_plot = df_rh.copy() # Original data
df_rh_plot = corrected_df_rh.copy() # Corrected data
df_rh_plot.index = pd.to_datetime(df_rh_plot.index)
df_rh_plot = df_rh_plot.resample('YE').mean()
df_rh_plot.index = df_rh_plot.index + pd.offsets.Day(1)
df_rh_plot.index = df_rh_plot.index - pd.offsets.YearBegin(1)
df_rh_plot = df_rh_plot.dropna(axis=1, how='all')

# Plot each column
for column in df_rh_plot.columns:
    plt.plot(df_rh_plot.index, df_rh_plot[column], label=column)

# Set major ticks to every 5 years
years = pd.date_range(start='2000', end='2100', freq='5YS')
plt.gca().set_xticks(years)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Add title and labels
#plt.title(f'RH Time Series Yearly average for {region}')
plt.title(f'Corrected RH Time Series Yearly average for {region}')
plt.xlabel('Year')
plt.ylabel('%')
plt.legend()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
           ncol=3, fancybox=True, shadow=True) # Legend below
plt.grid(True)
plt.tight_layout()
#plt.savefig(f'/home/users/clelland/Model/Analysis/CMIP and FWI time series/Ecoregion CSVs/{region}/rh_plot_{region}_2001_2100.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

### TP

#### Process and bias correction

In [ ]:
# tp
df_tp = pd.DataFrame({
    'ERA5-Land': df_e5l['tp'],
    'ACCESS_HIST': df_access_hist_climate['pr'],
    'ACCESS_SSP126': df_access_126_climate['tp'],
    'ACCESS_SSP245': df_access_245_climate['tp'],
    'ACCESS_SSP370': df_access_370_climate['tp'],
    'MRI_HIST': df_mri_hist_climate['pr'],
    'MRI_SSP126': df_mri_126_climate['tp'],
    'MRI_SSP245': df_mri_245_climate['tp'],
    'MRI_SSP370': df_mri_370_climate['tp'],
})

df_tp.sort_index(inplace=True)
df_tp

In [ ]:
# Bias correction
df_bias = df_tp.copy()

# First, ensure the index is datetime
df_bias.index = pd.to_datetime(df_bias.index)

# Add month as a new column
df_bias['month'] = df_bias.index.month

# Create a dictionary to store results
diff_results = {}

# 1. For 2001–2014 (historical)
hist_period = (df_bias.index >= '2001-01-01') & (df_bias.index <= '2014-12-31')

for model in ['ACCESS_HIST', 'MRI_HIST']:
    diff = df_bias.loc[hist_period, 'ERA5-Land'] - df_bias.loc[hist_period, model]
    monthly_diff = diff.groupby(df_bias.loc[hist_period, 'month']).mean()
    diff_results[model] = monthly_diff

# 2. For 2015–2023 (future scenarios)
future_period = (df_bias.index >= '2015-01-01') & (df_bias.index <= '2023-12-31')

for model in ['ACCESS_SSP126', 'ACCESS_SSP245', 'ACCESS_SSP370',
              'MRI_SSP126', 'MRI_SSP245', 'MRI_SSP370']:
    diff = df_bias.loc[future_period, 'ERA5-Land'] - df_bias.loc[future_period, model]
    monthly_diff = diff.groupby(df_bias.loc[future_period, 'month']).mean()
    diff_results[model] = monthly_diff

# 3. Combine all differences into a dataframe
diff_df = pd.DataFrame(diff_results)

# Optional: sort the dataframe by month
diff_df.index.name = 'month'
diff_df = diff_df.sort_index()

diff_df

In [ ]:
# 1. Define the period you want to correct
hist_correction_period = (df_bias.index >= '2001-01-01') & (df_bias.index <= '2014-12-31')
future_correction_period = (df_bias.index >= '2015-01-01') & (df_bias.index <= '2100-12-31')

# 2. Loop through each model/scenario you want to correct
corrected_data = {}
corrected_data['ERA5-Land'] = df_bias['ERA5-Land']

for model in ['ACCESS_HIST', 'MRI_HIST']:
    # Select the original model data for 2001–2014
    model_data = df_bias.loc[hist_correction_period, model].copy()
    
    # Get the corresponding month for each row
    months = df_bias.loc[hist_correction_period, 'month']
    
    # Subtract the bias correction based on the month
    corrected_values = model_data + months.map(diff_df[model])
    
    # Store in a dictionary
    corrected_data[model] = corrected_values

for model in ['ACCESS_SSP126', 'ACCESS_SSP245', 'ACCESS_SSP370',
              'MRI_SSP126', 'MRI_SSP245', 'MRI_SSP370']:
    # Select the original model data for 2015–2100
    model_data = df_bias.loc[future_correction_period, model].copy()
    
    # Get the corresponding month for each row
    months = df_bias.loc[future_correction_period, 'month']
    
    # Subtract the bias correction based on the month
    corrected_values = model_data + months.map(diff_df[model])
    
    # Store in a dictionary
    corrected_data[model] = corrected_values

# 3. Combine corrected data into a dataframe
corrected_df_tp = pd.DataFrame(corrected_data, index=df_bias.index)

corrected_df_tp

#### Plot

In [ ]:
df_tp_plot = df_tp.copy() # Original data
#df_tp_plot = corrected_df_tp.copy() # Corrected data
df_tp_plot.index = pd.to_datetime(df_tp_plot.index)
#df_tp_plot = df_tp_plot.loc['2001':'2014']
df_tp_plot = df_tp_plot.loc['2001':'2023']
df_tp_plot = df_tp_plot.dropna(axis=1, how='all')

# Set figure size
plt.figure(figsize=(14, 6))

# Plot each column
for column in df_tp_plot.columns:
    plt.plot(df_tp_plot.index, df_tp_plot[column], label=column)

# Set major ticks to every 5 years
#years = pd.date_range(start='2001', end='2015', freq='YS')
years = pd.date_range(start='2001', end='2023', freq='YS')
plt.gca().set_xticks(years)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Add title and labels
plt.title(f'TP Time Series for {region}')
#plt.title(f'Corrected TP Time Series for {region}')
plt.xlabel('Year')
plt.ylabel('m')
plt.legend()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
           ncol=3, fancybox=True, shadow=True) # Legend below
plt.grid(True)
plt.tight_layout()
#plt.savefig(f'/home/users/clelland/Model/Analysis/CMIP and FWI time series/Ecoregion CSVs/{region}/tp_plot_{region}_2001_2023.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
# Set figure size
plt.figure(figsize=(14, 6))

# Original data
#df_tp_plot = df_tp.copy() # Original data
df_tp_plot = corrected_df_tp.copy() # Corrected data
df_tp_plot.index = pd.to_datetime(df_tp_plot.index)
df_tp_plot = df_tp_plot.resample('YE').sum()
df_tp_plot.index = df_tp_plot.index + pd.offsets.Day(1)
df_tp_plot.index = df_tp_plot.index - pd.offsets.YearBegin(1)
df_tp_plot = df_tp_plot.dropna(axis=1, how='all')

# Plot first column (only up to 2024)
first_col = df_tp_plot.columns[0]
df_first = df_tp_plot.loc[:'2023-12-31', first_col]
plt.plot(df_first.index, df_first, label=first_col)

# Plot hist columns (only up to 2014)
hist_cols = df_tp_plot.columns[[1, 2]]
df_hist = df_tp_plot.loc[:'2014-12-31', hist_cols]
for col in hist_cols:
    plt.plot(df_hist.index, df_hist[col], label=col)
    
# Plot all other columns (from 2015 onwards)
future_cols = list(df_tp_plot.columns[3:])
for column in future_cols:
    df_other = df_tp_plot.loc['2015-01-01':, column]
    plt.plot(df_other.index, df_other, label=column)

# Set major ticks to every 5 years
years = pd.date_range(start='2000', end='2100', freq='5YS')
plt.gca().set_xticks(years)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Add title and labels
#plt.title(f'TP Time Series for {region}')
plt.title(f'Corrected TP Time Series for {region}')
plt.xlabel('Year')
plt.ylabel('m')
plt.legend()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
           ncol=3, fancybox=True, shadow=True) # Legend below
plt.grid(True)
plt.tight_layout()
#plt.savefig(f'/home/users/clelland/Model/Analysis/CMIP and FWI time series/Ecoregion CSVs/{region}/tp_plot_{region}_2001_2100.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

### RLDS

#### Process and bias correction

In [ ]:
# rlds
df_rlds = pd.DataFrame({
    'ERA5-Land': df_e5l['rlds'],
    'ACCESS_HIST': df_access_hist_climate['rlds'],
    'ACCESS_SSP126': df_access_126_climate['rlds'],
    'ACCESS_SSP245': df_access_245_climate['rlds'],
    'ACCESS_SSP370': df_access_370_climate['rlds'],
    'MRI_HIST': df_mri_hist_climate['rlds'],
    'MRI_SSP126': df_mri_126_climate['rlds'],
    'MRI_SSP245': df_mri_245_climate['rlds'],
    'MRI_SSP370': df_mri_370_climate['rlds'],
})

df_rlds.sort_index(inplace=True)
df_rlds

In [ ]:
# Bias correction
df_bias = df_rlds.copy()

# First, ensure the index is datetime
df_bias.index = pd.to_datetime(df_bias.index)

# Add month as a new column
df_bias['month'] = df_bias.index.month

# Create a dictionary to store results
diff_results = {}

# 1. For 2001–2014 (historical)
hist_period = (df_bias.index >= '2001-01-01') & (df_bias.index <= '2014-12-31')

for model in ['ACCESS_HIST', 'MRI_HIST']:
    diff = df_bias.loc[hist_period, 'ERA5-Land'] - df_bias.loc[hist_period, model]
    monthly_diff = diff.groupby(df_bias.loc[hist_period, 'month']).mean()
    diff_results[model] = monthly_diff

# 2. For 2015–2023 (future scenarios)
future_period = (df_bias.index >= '2015-01-01') & (df_bias.index <= '2023-12-31')

for model in ['ACCESS_SSP126', 'ACCESS_SSP245', 'ACCESS_SSP370',
              'MRI_SSP126', 'MRI_SSP245', 'MRI_SSP370']:
    diff = df_bias.loc[future_period, 'ERA5-Land'] - df_bias.loc[future_period, model]
    monthly_diff = diff.groupby(df_bias.loc[future_period, 'month']).mean()
    diff_results[model] = monthly_diff

# 3. Combine all differences into a dataframe
diff_df = pd.DataFrame(diff_results)

# Optional: sort the dataframe by month
diff_df.index.name = 'month'
diff_df = diff_df.sort_index()

diff_df

In [ ]:
# 1. Define the period you want to correct
hist_correction_period = (df_bias.index >= '2001-01-01') & (df_bias.index <= '2014-12-31')
future_correction_period = (df_bias.index >= '2015-01-01') & (df_bias.index <= '2100-12-31')

# 2. Loop through each model/scenario you want to correct
corrected_data = {}
corrected_data['ERA5-Land'] = df_bias['ERA5-Land']

for model in ['ACCESS_HIST', 'MRI_HIST']:
    # Select the original model data for 2001–2014
    model_data = df_bias.loc[hist_correction_period, model].copy()
    
    # Get the corresponding month for each row
    months = df_bias.loc[hist_correction_period, 'month']
    
    # Subtract the bias correction based on the month
    corrected_values = model_data + months.map(diff_df[model])
    
    # Store in a dictionary
    corrected_data[model] = corrected_values

for model in ['ACCESS_SSP126', 'ACCESS_SSP245', 'ACCESS_SSP370',
              'MRI_SSP126', 'MRI_SSP245', 'MRI_SSP370']:
    # Select the original model data for 2015–2100
    model_data = df_bias.loc[future_correction_period, model].copy()
    
    # Get the corresponding month for each row
    months = df_bias.loc[future_correction_period, 'month']
    
    # Subtract the bias correction based on the month
    corrected_values = model_data + months.map(diff_df[model])
    
    # Store in a dictionary
    corrected_data[model] = corrected_values

# 3. Combine corrected data into a dataframe
corrected_df_rlds = pd.DataFrame(corrected_data, index=df_bias.index)

corrected_df_rlds

#### Plot

In [ ]:
df_rlds_plot = df_rlds.copy() # Original data
#df_rlds_plot = corrected_df_rlds.copy() # Corrected data
df_rlds_plot.index = pd.to_datetime(df_rlds_plot.index)
#df_rlds_plot = df_rlds_plot.loc['2001':'2014']
df_rlds_plot = df_rlds_plot.loc['2001':'2023']
df_rlds_plot = df_rlds_plot.dropna(axis=1, how='all')

# Set figure size
plt.figure(figsize=(14, 6))

# Plot each column
for column in df_rlds_plot.columns:
    plt.plot(df_rlds_plot.index, df_rlds_plot[column], label=column)

# Set major ticks to every 5 years
#years = pd.date_range(start='2001', end='2015', freq='YS')
years = pd.date_range(start='2001', end='2023', freq='YS')
plt.gca().set_xticks(years)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Add title and labels
plt.title(f'RLDS Time Series for {region}')
#plt.title(f'Corrected RLDS Time Series for {region}')
plt.xlabel('Year')
plt.ylabel('W/m2')
plt.legend()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
           ncol=3, fancybox=True, shadow=True) # Legend below
plt.grid(True)
plt.tight_layout()
#plt.savefig(f'/home/users/clelland/Model/Analysis/CMIP and FWI time series/Ecoregion CSVs/{region}/rlds_plot_{region}_2001_2023.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
# Set figure size
plt.figure(figsize=(14, 6))

#df_rlds_plot = df_rlds.copy() # Original data
df_rlds_plot = corrected_df_rlds.copy() # Corrected data
df_rlds_plot.index = pd.to_datetime(df_rlds_plot.index)
df_rlds_plot = df_rlds_plot.resample('YE').mean()
df_rlds_plot.index = df_rlds_plot.index + pd.offsets.Day(1)
df_rlds_plot.index = df_rlds_plot.index - pd.offsets.YearBegin(1)
df_rlds_plot = df_rlds_plot.dropna(axis=1, how='all')

# Plot each column
for column in df_rlds_plot.columns:
    plt.plot(df_rlds_plot.index, df_rlds_plot[column], label=column)

# Set major ticks to every 5 years
years = pd.date_range(start='2000', end='2100', freq='5YS')
plt.gca().set_xticks(years)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Add title and labels
#plt.title(f'RLDS Time Series for {region}')
plt.title(f'Corrected RLDS Time Series for {region}')
plt.xlabel('Year')
plt.ylabel('W/m2')
plt.legend()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
           ncol=3, fancybox=True, shadow=True) # Legend below
plt.grid(True)
plt.tight_layout()
#plt.savefig(f'/home/users/clelland/Model/Analysis/CMIP and FWI time series/Ecoregion CSVs/{region}/rlds_plot_{region}_2001_2100.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

### RSDS

#### Process and bias correction

In [ ]:
# rsds
df_rsds = pd.DataFrame({
    'ERA5-Land': df_e5l['rsds'],
    'ACCESS_HIST': df_access_hist_climate['rsds'],
    'ACCESS_SSP126': df_access_126_climate['rsds'],
    'ACCESS_SSP245': df_access_245_climate['rsds'],
    'ACCESS_SSP370': df_access_370_climate['rsds'],
    'MRI_HIST': df_mri_hist_climate['rsds'],
    'MRI_SSP126': df_mri_126_climate['rsds'],
    'MRI_SSP245': df_mri_245_climate['rsds'],
    'MRI_SSP370': df_mri_370_climate['rsds'],
})

df_rsds.sort_index(inplace=True)
df_rsds

In [ ]:
# Bias correction
df_bias = df_rsds.copy()

# First, ensure the index is datetime
df_bias.index = pd.to_datetime(df_bias.index)

# Add month as a new column
df_bias['month'] = df_bias.index.month

# Create a dictionary to store results
diff_results = {}

# 1. For 2001–2014 (historical)
hist_period = (df_bias.index >= '2001-01-01') & (df_bias.index <= '2014-12-31')

for model in ['ACCESS_HIST', 'MRI_HIST']:
    diff = df_bias.loc[hist_period, 'ERA5-Land'] - df_bias.loc[hist_period, model]
    monthly_diff = diff.groupby(df_bias.loc[hist_period, 'month']).mean()
    diff_results[model] = monthly_diff

# 2. For 2015–2023 (future scenarios)
future_period = (df_bias.index >= '2015-01-01') & (df_bias.index <= '2023-12-31')

for model in ['ACCESS_SSP126', 'ACCESS_SSP245', 'ACCESS_SSP370',
              'MRI_SSP126', 'MRI_SSP245', 'MRI_SSP370']:
    diff = df_bias.loc[future_period, 'ERA5-Land'] - df_bias.loc[future_period, model]
    monthly_diff = diff.groupby(df_bias.loc[future_period, 'month']).mean()
    diff_results[model] = monthly_diff

# 3. Combine all differences into a dataframe
diff_df = pd.DataFrame(diff_results)

# Optional: sort the dataframe by month
diff_df.index.name = 'month'
diff_df = diff_df.sort_index()

diff_df

In [ ]:
# 1. Define the period you want to correct
hist_correction_period = (df_bias.index >= '2001-01-01') & (df_bias.index <= '2014-12-31')
future_correction_period = (df_bias.index >= '2015-01-01') & (df_bias.index <= '2100-12-31')

# 2. Loop through each model/scenario you want to correct
corrected_data = {}
corrected_data['ERA5-Land'] = df_bias['ERA5-Land']

for model in ['ACCESS_HIST', 'MRI_HIST']:
    # Select the original model data for 2001–2014
    model_data = df_bias.loc[hist_correction_period, model].copy()
    
    # Get the corresponding month for each row
    months = df_bias.loc[hist_correction_period, 'month']
    
    # Subtract the bias correction based on the month
    corrected_values = model_data + months.map(diff_df[model])
    
    # Store in a dictionary
    corrected_data[model] = corrected_values

for model in ['ACCESS_SSP126', 'ACCESS_SSP245', 'ACCESS_SSP370',
              'MRI_SSP126', 'MRI_SSP245', 'MRI_SSP370']:
    # Select the original model data for 2015–2100
    model_data = df_bias.loc[future_correction_period, model].copy()
    
    # Get the corresponding month for each row
    months = df_bias.loc[future_correction_period, 'month']
    
    # Subtract the bias correction based on the month
    corrected_values = model_data + months.map(diff_df[model])
    
    # Store in a dictionary
    corrected_data[model] = corrected_values

# 3. Combine corrected data into a dataframe
corrected_df_rsds = pd.DataFrame(corrected_data, index=df_bias.index)

corrected_df_rsds

#### Plot

In [ ]:
#df_rsds_plot = df_rsds.copy() # Original data
df_rsds_plot = corrected_df_rsds.copy() # Corrected data
df_rsds_plot.index = pd.to_datetime(df_rsds_plot.index)
#df_rsds_plot = df_rsds_plot.loc['2001':'2014']
df_rsds_plot = df_rsds_plot.loc['2001':'2023']
df_rsds_plot = df_rsds_plot.dropna(axis=1, how='all')

# Set figure size
plt.figure(figsize=(14, 6))

# Plot each column
for column in df_rsds_plot.columns:
    plt.plot(df_rsds_plot.index, df_rsds_plot[column], label=column)

# Set major ticks to every 5 years
years = pd.date_range(start='2001', end='2015', freq='YS')
#years = pd.date_range(start='2001', end='2023', freq='YS')
plt.gca().set_xticks(years)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Add title and labels
#plt.title(f'RSDS Time Series for {region}')
plt.title(f'Corrected RSDS Time Series for {region}')
plt.xlabel('Year')
plt.ylabel('W/m2')
plt.legend()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
           ncol=3, fancybox=True, shadow=True) # Legend below
plt.grid(True)
plt.tight_layout()
#plt.savefig(f'/home/users/clelland/Model/Analysis/CMIP and FWI time series/Ecoregion CSVs/{region}/rsds_plot_{region}_2001_2023.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
# Set figure size
plt.figure(figsize=(14, 6))

#df_rsds_plot = df_rsds.copy() # Original data
df_rsds_plot = corrected_df_rsds.copy() # Corrected data
df_rsds_plot.index = pd.to_datetime(df_rsds_plot.index)
df_rsds_plot = df_rsds_plot.resample('YE').mean()
df_rsds_plot.index = df_rsds_plot.index + pd.offsets.Day(1)
df_rsds_plot.index = df_rsds_plot.index - pd.offsets.YearBegin(1)
df_rsds_plot = df_rsds_plot.dropna(axis=1, how='all')

# Plot each column
for column in df_rsds_plot.columns:
    plt.plot(df_rsds_plot.index, df_rsds_plot[column], label=column)

# Set major ticks to every 5 years
years = pd.date_range(start='2000', end='2100', freq='5YS')
plt.gca().set_xticks(years)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Add title and labels
#plt.title(f'RSDS Time Series for {region}')
plt.title(f'Corrected RSDS Time Series for {region}')
plt.xlabel('Year')
plt.ylabel('W/m2')
plt.legend()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
           ncol=3, fancybox=True, shadow=True) # Legend below
plt.grid(True)
plt.tight_layout()
#plt.savefig(f'/home/users/clelland/Model/Analysis/CMIP and FWI time series/Ecoregion CSVs/{region}/rsds_plot_{region}_2001_2100.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

### WSP

#### Process and bias correction

In [ ]:
# wsp
df_wsp = pd.DataFrame({
    'ERA5-Land': df_e5l['wsp'],
    'ACCESS_HIST': df_access_hist_climate['sfcWind'],
    'ACCESS_SSP126': df_access_126_climate['wsp'],
    'ACCESS_SSP245': df_access_245_climate['wsp'],
    'ACCESS_SSP370': df_access_370_climate['wsp'],
    'MRI_HIST': df_mri_hist_climate['sfcWind'],
    'MRI_SSP126': df_mri_126_climate['wsp'],
    'MRI_SSP245': df_mri_245_climate['wsp'],
    'MRI_SSP370': df_mri_370_climate['wsp'],
})

df_wsp.sort_index(inplace=True)
df_wsp

In [ ]:
# Bias correction
df_bias = df_wsp.copy()

# First, ensure the index is datetime
df_bias.index = pd.to_datetime(df_bias.index)

# Add month as a new column
df_bias['month'] = df_bias.index.month

# Create a dictionary to store results
diff_results = {}

# 1. For 2001–2014 (historical)
hist_period = (df_bias.index >= '2001-01-01') & (df_bias.index <= '2014-12-31')

for model in ['ACCESS_HIST', 'MRI_HIST']:
    diff = df_bias.loc[hist_period, 'ERA5-Land'] - df_bias.loc[hist_period, model]
    monthly_diff = diff.groupby(df_bias.loc[hist_period, 'month']).mean()
    diff_results[model] = monthly_diff

# 2. For 2015–2023 (future scenarios)
future_period = (df_bias.index >= '2015-01-01') & (df_bias.index <= '2023-12-31')

for model in ['ACCESS_SSP126', 'ACCESS_SSP245', 'ACCESS_SSP370',
              'MRI_SSP126', 'MRI_SSP245', 'MRI_SSP370']:
    diff = df_bias.loc[future_period, 'ERA5-Land'] - df_bias.loc[future_period, model]
    monthly_diff = diff.groupby(df_bias.loc[future_period, 'month']).mean()
    diff_results[model] = monthly_diff

# 3. Combine all differences into a dataframe
diff_df = pd.DataFrame(diff_results)

# Optional: sort the dataframe by month
diff_df.index.name = 'month'
diff_df = diff_df.sort_index()

diff_df

In [ ]:
# 1. Define the period you want to correct
hist_correction_period = (df_bias.index >= '2001-01-01') & (df_bias.index <= '2014-12-31')
future_correction_period = (df_bias.index >= '2015-01-01') & (df_bias.index <= '2100-12-31')

# 2. Loop through each model/scenario you want to correct
corrected_data = {}
corrected_data['ERA5-Land'] = df_bias['ERA5-Land']

for model in ['ACCESS_HIST', 'MRI_HIST']:
    # Select the original model data for 2001–2014
    model_data = df_bias.loc[hist_correction_period, model].copy()
    
    # Get the corresponding month for each row
    months = df_bias.loc[hist_correction_period, 'month']
    
    # Subtract the bias correction based on the month
    corrected_values = model_data + months.map(diff_df[model])
    
    # Store in a dictionary
    corrected_data[model] = corrected_values

for model in ['ACCESS_SSP126', 'ACCESS_SSP245', 'ACCESS_SSP370',
              'MRI_SSP126', 'MRI_SSP245', 'MRI_SSP370']:
    # Select the original model data for 2015–2100
    model_data = df_bias.loc[future_correction_period, model].copy()
    
    # Get the corresponding month for each row
    months = df_bias.loc[future_correction_period, 'month']
    
    # Subtract the bias correction based on the month
    corrected_values = model_data + months.map(diff_df[model])
    
    # Store in a dictionary
    corrected_data[model] = corrected_values

# 3. Combine corrected data into a dataframe
corrected_df_wsp = pd.DataFrame(corrected_data, index=df_bias.index)

corrected_df_wsp

#### Plot

In [ ]:
#df_wsp_plot = df_wsp.copy() # Original data
df_wsp_plot = corrected_df_wsp.copy() # Corrected data
df_wsp_plot.index = pd.to_datetime(df_wsp_plot.index)
#df_wsp_plot = df_wsp_plot.loc['2001':'2014']
df_wsp_plot = df_wsp_plot.loc['2001':'2023']
df_wsp_plot = df_wsp_plot.dropna(axis=1, how='all')

# Set figure size
plt.figure(figsize=(14, 6))

# Plot each column
for column in df_wsp_plot.columns:
    plt.plot(df_wsp_plot.index, df_wsp_plot[column], label=column)

# Set major ticks to every 5 years
#years = pd.date_range(start='2001', end='2015', freq='YS')
years = pd.date_range(start='2001', end='2023', freq='2YS')
plt.gca().set_xticks(years)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Add title and labels
#plt.title(f'Original WSP Time Series for {region}')
plt.title(f'Bias Corrected WSP Time Series for {region}')
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlabel('Year', fontsize=18)
plt.ylabel('m/s', fontsize=18)
plt.legend()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
           ncol=3, fancybox=True, shadow=True) # Legend below
plt.grid(True)
plt.ylim(0, 4.2)
plt.tight_layout()
#plt.savefig(f'/home/users/clelland/Model/Analysis/CMIP and FWI time series/Ecoregion CSVs/{region}/wsp_plot_{region}_2001_2023.png', dpi=300, bbox_inches='tight', transparent=True)
#plt.savefig('/home/users/clelland/Model/Analysis/wsp_midbor_new_short', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
# Set figure size
plt.figure(figsize=(14, 6))

#df_wsp_plot = df_wsp.copy() # Original data
df_wsp_plot = corrected_df_wsp.copy() # Corrected data
df_wsp_plot.index = pd.to_datetime(df_wsp_plot.index)
df_wsp_plot = df_wsp_plot.resample('YE').mean()
df_wsp_plot.index = df_wsp_plot.index + pd.offsets.Day(1)
df_wsp_plot.index = df_wsp_plot.index - pd.offsets.YearBegin(1)
df_wsp_plot = df_wsp_plot.dropna(axis=1, how='all')

# Plot each column
for column in df_wsp_plot.columns:
    plt.plot(df_wsp_plot.index, df_wsp_plot[column], label=column)

# Set major ticks to every 5 years
years = pd.date_range(start='2000', end='2100', freq='10YS')
plt.gca().set_xticks(years)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Add title and labels
#plt.title(f'Original WSP Time Series for {region}')
plt.title(f'Bias Corrected WSP Time Series for {region}')
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlabel('Year', fontsize=18)
plt.ylabel('m/s', fontsize=18)
plt.legend()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
           ncol=3, fancybox=True, shadow=True) # Legend below
plt.grid(True)
plt.ylim(0, 4.2)
plt.tight_layout()
#plt.savefig(f'/home/users/clelland/Model/Analysis/CMIP and FWI time series/Ecoregion CSVs/{region}/wsp_plot_{region}_2001_2100.png', dpi=300, bbox_inches='tight', transparent=True)
#plt.savefig('/home/users/clelland/Model/Analysis/wsp_midbor_new_long', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

### T2M

#### Process and bias correction

In [ ]:
# t2m
df_t2m = pd.DataFrame({
    'ERA5-Land': df_e5l['t2m'],
    'ACCESS_HIST': df_access_hist_climate['tas'],
    'ACCESS_SSP126': df_access_126_climate['t2m'],
    'ACCESS_SSP245': df_access_245_climate['t2m'],
    'ACCESS_SSP370': df_access_370_climate['t2m'],
    'MRI_HIST': df_mri_hist_climate['tas'],
    'MRI_SSP126': df_mri_126_climate['t2m'],
    'MRI_SSP245': df_mri_245_climate['t2m'],
    'MRI_SSP370': df_mri_370_climate['t2m'],
})

df_t2m.sort_index(inplace=True)
df_t2m

In [ ]:
# Bias correction
df_bias = df_t2m.copy()

# First, ensure the index is datetime
df_bias.index = pd.to_datetime(df_bias.index)

# Add month as a new column
df_bias['month'] = df_bias.index.month

# Create a dictionary to store results
diff_results = {}

# 1. For 2001–2014 (historical)
hist_period = (df_bias.index >= '2001-01-01') & (df_bias.index <= '2014-12-31')

for model in ['ACCESS_HIST', 'MRI_HIST']:
    diff = df_bias.loc[hist_period, 'ERA5-Land'] - df_bias.loc[hist_period, model]
    monthly_diff = diff.groupby(df_bias.loc[hist_period, 'month']).mean()
    diff_results[model] = monthly_diff

# 2. For 2015–2023 (future scenarios)
future_period = (df_bias.index >= '2015-01-01') & (df_bias.index <= '2023-12-31')

for model in ['ACCESS_SSP126', 'ACCESS_SSP245', 'ACCESS_SSP370',
              'MRI_SSP126', 'MRI_SSP245', 'MRI_SSP370']:
    diff = df_bias.loc[future_period, 'ERA5-Land'] - df_bias.loc[future_period, model]
    monthly_diff = diff.groupby(df_bias.loc[future_period, 'month']).mean()
    diff_results[model] = monthly_diff

# 3. Combine all differences into a dataframe
diff_df = pd.DataFrame(diff_results)

# Optional: sort the dataframe by month
diff_df.index.name = 'month'
diff_df = diff_df.sort_index()

diff_df

In [ ]:
# 1. Define the period you want to correct
hist_correction_period = (df_bias.index >= '2001-01-01') & (df_bias.index <= '2014-12-31')
future_correction_period = (df_bias.index >= '2015-01-01') & (df_bias.index <= '2100-12-31')

# 2. Loop through each model/scenario you want to correct
corrected_data = {}
corrected_data['ERA5-Land'] = df_bias['ERA5-Land']

for model in ['ACCESS_HIST', 'MRI_HIST']:
    # Select the original model data for 2001–2014
    model_data = df_bias.loc[hist_correction_period, model].copy()
    
    # Get the corresponding month for each row
    months = df_bias.loc[hist_correction_period, 'month']
    
    # Subtract the bias correction based on the month
    corrected_values = model_data + months.map(diff_df[model])
    
    # Store in a dictionary
    corrected_data[model] = corrected_values

for model in ['ACCESS_SSP126', 'ACCESS_SSP245', 'ACCESS_SSP370',
              'MRI_SSP126', 'MRI_SSP245', 'MRI_SSP370']:
    # Select the original model data for 2015–2100
    model_data = df_bias.loc[future_correction_period, model].copy()
    
    # Get the corresponding month for each row
    months = df_bias.loc[future_correction_period, 'month']
    
    # Subtract the bias correction based on the month
    corrected_values = model_data + months.map(diff_df[model])
    
    # Store in a dictionary
    corrected_data[model] = corrected_values

# 3. Combine corrected data into a dataframe
corrected_df_t2m = pd.DataFrame(corrected_data, index=df_bias.index)

corrected_df_t2m

#### Plot

In [ ]:
df_t2m_plot = df_t2m.copy() # Original data
#df_t2m_plot = corrected_df_t2m.copy() # Corrected data
df_t2m_plot.index = pd.to_datetime(df_t2m_plot.index)
#df_t2m_plot = df_t2m_plot.loc['2001':'2014']
df_t2m_plot = df_t2m_plot.loc['2001':'2023']
df_t2m_plot = df_t2m_plot.dropna(axis=1, how='all')

# Set figure size
plt.figure(figsize=(14, 6))

# Plot each column
for column in df_t2m_plot.columns:
    plt.plot(df_t2m_plot.index, df_t2m_plot[column], label=column)

# Set major ticks to every 5 years
#years = pd.date_range(start='2001', end='2015', freq='YS')
years = pd.date_range(start='2001', end='2023', freq='YS')
plt.gca().set_xticks(years)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Add title and labels
#plt.title(f'T2M Time Series for {region}')
plt.title(f'Corrected T2M Time Series for {region}')
plt.xlabel('Year')
plt.ylabel('K')
plt.legend()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
           ncol=3, fancybox=True, shadow=True) # Legend below
plt.grid(True)
plt.tight_layout()
#plt.savefig(f'/home/users/clelland/Model/Analysis/CMIP and FWI time series/Ecoregion CSVs/{region}/t2m_plot_{region}_2001_2023.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
# Set figure size
plt.figure(figsize=(14, 6))

#df_t2m_plot = df_t2m.copy() # Original data
#df_t2m_plot = corrected_df_t2m.copy() # Corrected data
df_t2m_plot = corrected_df_t2m.copy().loc['2025':] # Corrected data
df_t2m_plot.index = pd.to_datetime(df_t2m_plot.index)
df_t2m_plot = df_t2m_plot.resample('YE').mean()
df_t2m_plot.index = df_t2m_plot.index + pd.offsets.Day(1)
df_t2m_plot.index = df_t2m_plot.index - pd.offsets.YearBegin(1)
df_t2m_plot = df_t2m_plot.dropna(axis=1, how='all')

# Plot each column
for column in df_t2m_plot.columns:
    plt.plot(df_t2m_plot.index, df_t2m_plot[column], label=column)

# Set major ticks to every 5 years
#years = pd.date_range(start='2000', end='2100', freq='5YS')
years = pd.date_range(start='2025', end='2100', freq='5YS')
plt.gca().set_xticks(years)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Add title and labels
#plt.title(f'T2M Time Series for {region}')
plt.title(f'Corrected T2M Time Series for {region}')
plt.xlabel('Year')
plt.ylabel('K')
plt.legend()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
           ncol=3, fancybox=True, shadow=True) # Legend below
plt.grid(True)
plt.tight_layout()
#plt.savefig(f'/home/users/clelland/Model/Analysis/CMIP and FWI time series/Ecoregion CSVs/{region}/t2m_plot_{region}_2001_2100.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

### MX2T

#### Process and bias correction

In [ ]:
# mx2t
df_mx2t = pd.DataFrame({
    'ERA5-Land': df_e5l['mx2t'],
    'ACCESS_HIST': df_access_hist_climate['tasmax'],
    'ACCESS_SSP126': df_access_126_climate['mx2t'],
    'ACCESS_SSP245': df_access_245_climate['mx2t'],
    'ACCESS_SSP370': df_access_370_climate['mx2t'],
    'MRI_HIST': df_mri_hist_climate['tasmax'],
    'MRI_SSP126': df_mri_126_climate['mx2t'],
    'MRI_SSP245': df_mri_245_climate['mx2t'],
    'MRI_SSP370': df_mri_370_climate['mx2t'],
})

df_mx2t.sort_index(inplace=True)
df_mx2t

In [ ]:
# Bias correction
df_bias = df_mx2t.copy()

# First, ensure the index is datetime
df_bias.index = pd.to_datetime(df_bias.index)

# Add month as a new column
df_bias['month'] = df_bias.index.month

# Create a dictionary to store results
diff_results = {}

# 1. For 2001–2014 (historical)
hist_period = (df_bias.index >= '2001-01-01') & (df_bias.index <= '2014-12-31')

for model in ['ACCESS_HIST', 'MRI_HIST']:
    diff = df_bias.loc[hist_period, 'ERA5-Land'] - df_bias.loc[hist_period, model]
    monthly_diff = diff.groupby(df_bias.loc[hist_period, 'month']).mean()
    diff_results[model] = monthly_diff

# 2. For 2015–2023 (future scenarios)
future_period = (df_bias.index >= '2015-01-01') & (df_bias.index <= '2023-12-31')

for model in ['ACCESS_SSP126', 'ACCESS_SSP245', 'ACCESS_SSP370',
              'MRI_SSP126', 'MRI_SSP245', 'MRI_SSP370']:
    diff = df_bias.loc[future_period, 'ERA5-Land'] - df_bias.loc[future_period, model]
    monthly_diff = diff.groupby(df_bias.loc[future_period, 'month']).mean()
    diff_results[model] = monthly_diff

# 3. Combine all differences into a dataframe
diff_df = pd.DataFrame(diff_results)

# Optional: sort the dataframe by month
diff_df.index.name = 'month'
diff_df = diff_df.sort_index()

diff_df

In [ ]:
# 1. Define the period you want to correct
hist_correction_period = (df_bias.index >= '2001-01-01') & (df_bias.index <= '2014-12-31')
future_correction_period = (df_bias.index >= '2015-01-01') & (df_bias.index <= '2100-12-31')

# 2. Loop through each model/scenario you want to correct
corrected_data = {}
corrected_data['ERA5-Land'] = df_bias['ERA5-Land']

for model in ['ACCESS_HIST', 'MRI_HIST']:
    # Select the original model data for 2001–2014
    model_data = df_bias.loc[hist_correction_period, model].copy()
    
    # Get the corresponding month for each row
    months = df_bias.loc[hist_correction_period, 'month']
    
    # Subtract the bias correction based on the month
    corrected_values = model_data + months.map(diff_df[model])
    
    # Store in a dictionary
    corrected_data[model] = corrected_values

for model in ['ACCESS_SSP126', 'ACCESS_SSP245', 'ACCESS_SSP370',
              'MRI_SSP126', 'MRI_SSP245', 'MRI_SSP370']:
    # Select the original model data for 2015–2100
    model_data = df_bias.loc[future_correction_period, model].copy()
    
    # Get the corresponding month for each row
    months = df_bias.loc[future_correction_period, 'month']
    
    # Subtract the bias correction based on the month
    corrected_values = model_data + months.map(diff_df[model])
    
    # Store in a dictionary
    corrected_data[model] = corrected_values

# 3. Combine corrected data into a dataframe
corrected_df_mx2t = pd.DataFrame(corrected_data, index=df_bias.index)

corrected_df_mx2t

#### Plot

In [ ]:
#df_mx2t_plot = df_mx2t.copy() # Original data
df_mx2t_plot = corrected_df_mx2t.copy() # Corrected data
df_mx2t_plot.index = pd.to_datetime(df_mx2t_plot.index)
#df_mx2t_plot = df_mx2t_plot.loc['2001':'2014']
df_mx2t_plot = df_mx2t_plot.loc['2001':'2023']
df_mx2t_plot = df_mx2t_plot.dropna(axis=1, how='all')

# Set figure size
plt.figure(figsize=(14, 6))

# Plot each column
for column in df_mx2t_plot.columns:
    plt.plot(df_mx2t_plot.index, df_mx2t_plot[column], label=column)

# Set major ticks to every 5 years
#years = pd.date_range(start='2001', end='2015', freq='YS')
years = pd.date_range(start='2001', end='2023', freq='YS')
plt.gca().set_xticks(years)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Add title and labels
#plt.title(f'MX2T Time Series for {region}')
plt.title(f'Corrected MX2T Time Series for {region}')
plt.xlabel('Year')
plt.ylabel('K')
plt.legend()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
           ncol=3, fancybox=True, shadow=True) # Legend below
plt.grid(True)
plt.tight_layout()
#plt.savefig(f'/home/users/clelland/Model/Analysis/CMIP and FWI time series/Ecoregion CSVs/{region}/mx2t_plot_{region}_2001_2023.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
# Set figure size
plt.figure(figsize=(14, 6))

#df_mx2t_plot = df_mx2t.copy() # Original data
df_mx2t_plot = corrected_df_mx2t.copy() # Corrected data
df_mx2t_plot.index = pd.to_datetime(df_mx2t_plot.index)
df_mx2t_plot = df_mx2t_plot.resample('YE').mean()
df_mx2t_plot.index = df_mx2t_plot.index + pd.offsets.Day(1)
df_mx2t_plot.index = df_mx2t_plot.index - pd.offsets.YearBegin(1)
df_mx2t_plot = df_mx2t_plot.dropna(axis=1, how='all')

# Plot each column
for column in df_mx2t_plot.columns:
    plt.plot(df_mx2t_plot.index, df_mx2t_plot[column], label=column)

# Set major ticks to every 5 years
years = pd.date_range(start='2000', end='2100', freq='5YS')
plt.gca().set_xticks(years)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Add title and labels
#plt.title(f'MX2T Time Series for {region}')
plt.title(f'Corrected MX2T Time Series for {region}')
plt.xlabel('Year')
plt.ylabel('K')
plt.legend()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
           ncol=3, fancybox=True, shadow=True) # Legend below
plt.grid(True)
plt.tight_layout()
#plt.savefig(f'/home/users/clelland/Model/Analysis/CMIP and FWI time series/Ecoregion CSVs/{region}/mx2t_plot_{region}_2001_2100.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

### MN2T

#### Process and bias correction

In [ ]:
# mn2t
df_mn2t = pd.DataFrame({
    'ERA5-Land': df_e5l['mn2t'],
    'ACCESS_HIST': df_access_hist_climate['tasmin'],
    'ACCESS_SSP126': df_access_126_climate['mn2t'],
    'ACCESS_SSP245': df_access_245_climate['mn2t'],
    'ACCESS_SSP370': df_access_370_climate['mn2t'],
    'MRI_HIST': df_mri_hist_climate['tasmin'],
    'MRI_SSP126': df_mri_126_climate['mn2t'],
    'MRI_SSP245': df_mri_245_climate['mn2t'],
    'MRI_SSP370': df_mri_370_climate['mn2t'],
})

df_mn2t.sort_index(inplace=True)
df_mn2t

In [ ]:
# Bias correction
df_bias = df_mn2t.copy()

# First, ensure the index is datetime
df_bias.index = pd.to_datetime(df_bias.index)

# Add month as a new column
df_bias['month'] = df_bias.index.month

# Create a dictionary to store results
diff_results = {}

# 1. For 2001–2014 (historical)
hist_period = (df_bias.index >= '2001-01-01') & (df_bias.index <= '2014-12-31')

for model in ['ACCESS_HIST', 'MRI_HIST']:
    diff = df_bias.loc[hist_period, 'ERA5-Land'] - df_bias.loc[hist_period, model]
    monthly_diff = diff.groupby(df_bias.loc[hist_period, 'month']).mean()
    diff_results[model] = monthly_diff

# 2. For 2015–2023 (future scenarios)
future_period = (df_bias.index >= '2015-01-01') & (df_bias.index <= '2023-12-31')

for model in ['ACCESS_SSP126', 'ACCESS_SSP245', 'ACCESS_SSP370',
              'MRI_SSP126', 'MRI_SSP245', 'MRI_SSP370']:
    diff = df_bias.loc[future_period, 'ERA5-Land'] - df_bias.loc[future_period, model]
    monthly_diff = diff.groupby(df_bias.loc[future_period, 'month']).mean()
    diff_results[model] = monthly_diff

# 3. Combine all differences into a dataframe
diff_df = pd.DataFrame(diff_results)

# Optional: sort the dataframe by month
diff_df.index.name = 'month'
diff_df = diff_df.sort_index()

diff_df

In [ ]:
# 1. Define the period you want to correct
hist_correction_period = (df_bias.index >= '2001-01-01') & (df_bias.index <= '2014-12-31')
future_correction_period = (df_bias.index >= '2015-01-01') & (df_bias.index <= '2100-12-31')

# 2. Loop through each model/scenario you want to correct
corrected_data = {}
corrected_data['ERA5-Land'] = df_bias['ERA5-Land']

for model in ['ACCESS_HIST', 'MRI_HIST']:
    # Select the original model data for 2001–2014
    model_data = df_bias.loc[hist_correction_period, model].copy()
    
    # Get the corresponding month for each row
    months = df_bias.loc[hist_correction_period, 'month']
    
    # Subtract the bias correction based on the month
    corrected_values = model_data + months.map(diff_df[model])
    
    # Store in a dictionary
    corrected_data[model] = corrected_values

for model in ['ACCESS_SSP126', 'ACCESS_SSP245', 'ACCESS_SSP370',
              'MRI_SSP126', 'MRI_SSP245', 'MRI_SSP370']:
    # Select the original model data for 2015–2100
    model_data = df_bias.loc[future_correction_period, model].copy()
    
    # Get the corresponding month for each row
    months = df_bias.loc[future_correction_period, 'month']
    
    # Subtract the bias correction based on the month
    corrected_values = model_data + months.map(diff_df[model])
    
    # Store in a dictionary
    corrected_data[model] = corrected_values

# 3. Combine corrected data into a dataframe
corrected_df_mn2t = pd.DataFrame(corrected_data, index=df_bias.index)

corrected_df_mn2t

#### Plot

In [ ]:
#df_mn2t_plot = df_mn2t.copy() # Original data
df_mn2t_plot = corrected_df_mn2t.copy() # Corrected data
df_mn2t_plot.index = pd.to_datetime(df_mn2t_plot.index)
#df_mn2t_plot = df_mn2t_plot.loc['2001':'2014']
df_mn2t_plot = df_mn2t_plot.loc['2001':'2023']
df_mn2t_plot = df_mn2t_plot.dropna(axis=1, how='all')

# Set figure size
plt.figure(figsize=(14, 6))

# Plot each column
for column in df_mn2t_plot.columns:
    plt.plot(df_mn2t_plot.index, df_mn2t_plot[column], label=column)

# Set major ticks to every 5 years
#years = pd.date_range(start='2001', end='2015', freq='YS')
years = pd.date_range(start='2001', end='2023', freq='YS')
plt.gca().set_xticks(years)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Add title and labels
#plt.title(f'MN2T Time Series for {region}')
plt.title(f'Corrected MN2T Time Series for {region}')
plt.xlabel('Year')
plt.ylabel('K')
plt.legend()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
           ncol=3, fancybox=True, shadow=True) # Legend below
plt.grid(True)
plt.tight_layout()
#plt.savefig(f'/home/users/clelland/Model/Analysis/CMIP and FWI time series/Ecoregion CSVs/{region}/mn2t_plot_{region}_2001_2023.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
# Set figure size
plt.figure(figsize=(14, 6))

#df_mn2t_plot = df_mn2t.copy() # Original data
df_mn2t_plot = corrected_df_mn2t.copy() # Corrected data
df_mn2t_plot.index = pd.to_datetime(df_mn2t_plot.index)
df_mn2t_plot = df_mn2t_plot.resample('YE').mean()
df_mn2t_plot.index = df_mn2t_plot.index + pd.offsets.Day(1)
df_mn2t_plot.index = df_mn2t_plot.index - pd.offsets.YearBegin(1)
df_mn2t_plot = df_mn2t_plot.dropna(axis=1, how='all')

# Plot each column
for column in df_mn2t_plot.columns:
    plt.plot(df_mn2t_plot.index, df_mn2t_plot[column], label=column)

# Set major ticks to every 5 years
years = pd.date_range(start='2000', end='2100', freq='5YS')
plt.gca().set_xticks(years)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Add title and labels
#plt.title(f'MN2T Time Series for {region}')
plt.title(f'Corrected MN2T Time Series for {region}')
plt.xlabel('Year')
plt.ylabel('K')
plt.legend()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
           ncol=3, fancybox=True, shadow=True) # Legend below
plt.grid(True)
plt.tight_layout()
#plt.savefig(f'/home/users/clelland/Model/Analysis/CMIP and FWI time series/Ecoregion CSVs/{region}/mn2t_plot_{region}_2001_2100.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

## FWI

### BUI

#### Process and bias correction

In [ ]:
# BUI
df_bui = pd.DataFrame({
    'CEMS': df_cems['BUI'],
    'ACCESS_HIST': df_access_hist_fwi['BUI'],
    'ACCESS_SSP126': df_access_126_fwi['BUI'],
    'ACCESS_SSP245': df_access_245_fwi['BUI'],
    'ACCESS_SSP370': df_access_370_fwi['BUI'],
    'MRI_HIST': df_mri_hist_fwi['BUI'],
    'MRI_SSP126': df_mri_126_fwi['BUI'],
    'MRI_SSP245': df_mri_245_fwi['BUI'],
    'MRI_SSP370': df_mri_370_fwi['BUI'],
})

df_bui.sort_index(inplace=True)
df_bui

In [ ]:
# Bias correction
df_bias = df_bui.copy()

# First, ensure the index is datetime
df_bias.index = pd.to_datetime(df_bias.index)

# Add month as a new column
df_bias['month'] = df_bias.index.month

# Create a dictionary to store results
diff_results = {}

# 1. For 2001–2014 (historical)
hist_period = (df_bias.index >= '2001-01-01') & (df_bias.index <= '2014-12-31')

for model in ['ACCESS_HIST', 'MRI_HIST']:
    diff = df_bias.loc[hist_period, 'CEMS'] - df_bias.loc[hist_period, model]
    monthly_diff = diff.groupby(df_bias.loc[hist_period, 'month']).mean()
    diff_results[model] = monthly_diff

# 2. For 2015–2023 (future scenarios)
future_period = (df_bias.index >= '2015-01-01') & (df_bias.index <= '2023-12-31')

for model in ['ACCESS_SSP126', 'ACCESS_SSP245', 'ACCESS_SSP370',
              'MRI_SSP126', 'MRI_SSP245', 'MRI_SSP370']:
    diff = df_bias.loc[future_period, 'CEMS'] - df_bias.loc[future_period, model]
    monthly_diff = diff.groupby(df_bias.loc[future_period, 'month']).mean()
    diff_results[model] = monthly_diff

# 3. Combine all differences into a dataframe
diff_df = pd.DataFrame(diff_results)

# Optional: sort the dataframe by month
diff_df.index.name = 'month'
diff_df = diff_df.sort_index()

diff_df

In [ ]:
# 1. Define the period you want to correct
hist_correction_period = (df_bias.index >= '2001-01-01') & (df_bias.index <= '2014-12-31')
future_correction_period = (df_bias.index >= '2015-01-01') & (df_bias.index <= '2100-12-31')

# 2. Loop through each model/scenario you want to correct
corrected_data = {}
corrected_data['CEMS'] = df_bias['CEMS']

for model in ['ACCESS_HIST', 'MRI_HIST']:
    # Select the original model data for 2001–2014
    model_data = df_bias.loc[hist_correction_period, model].copy()
    
    # Get the corresponding month for each row
    months = df_bias.loc[hist_correction_period, 'month']
    
    # Subtract the bias correction based on the month
    corrected_values = model_data + months.map(diff_df[model])
    
    # Store in a dictionary
    corrected_data[model] = corrected_values

for model in ['ACCESS_SSP126', 'ACCESS_SSP245', 'ACCESS_SSP370',
              'MRI_SSP126', 'MRI_SSP245', 'MRI_SSP370']:
    # Select the original model data for 2015–2100
    model_data = df_bias.loc[future_correction_period, model].copy()
    
    # Get the corresponding month for each row
    months = df_bias.loc[future_correction_period, 'month']
    
    # Subtract the bias correction based on the month
    corrected_values = model_data + months.map(diff_df[model])
    
    # Store in a dictionary
    corrected_data[model] = corrected_values

# 3. Combine corrected data into a dataframe
corrected_df_bui = pd.DataFrame(corrected_data, index=df_bias.index)

corrected_df_bui

#### Plot

In [ ]:
#df_bui_plot = df_bui.copy() # Original data
df_bui_plot = corrected_df_bui.copy() # Corrected data
df_bui_plot.index = pd.to_datetime(df_bui_plot.index)
#df_bui_plot = df_bui_plot.loc['2001':'2014']
df_bui_plot = df_bui_plot.loc['2001':'2023']
df_bui_plot = df_bui_plot.dropna(axis=1, how='all')

# Set figure size
plt.figure(figsize=(14, 6))

# Plot each column
for column in df_bui_plot.columns:
    plt.plot(df_bui_plot.index, df_bui_plot[column], label=column)

# Set major ticks to every 5 years
#years = pd.date_range(start='2001', end='2015', freq='YS')
years = pd.date_range(start='2001', end='2023', freq='YS')
plt.gca().set_xticks(years)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Add title and labels
#plt.title(f'BUI Time Series for {region}')
plt.title(f'Corrected BUI Time Series for {region}')
plt.xlabel('Year')
plt.ylabel('BUI')
plt.legend()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
           ncol=3, fancybox=True, shadow=True) # Legend below
plt.grid(True)
plt.tight_layout()
#plt.savefig(f'/home/users/clelland/Model/Analysis/CMIP and FWI time series/Ecoregion CSVs/{region}/bui_plot_{region}_2001_2023.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
# Set figure size
plt.figure(figsize=(14, 6))

#df_bui_plot = df_bui.copy() # Original data
df_bui_plot = corrected_df_bui.copy() # Corrected data
df_bui_plot.index = pd.to_datetime(df_bui_plot.index)
df_bui_plot = df_bui_plot.resample('YE').mean()
df_bui_plot.index = df_bui_plot.index + pd.offsets.Day(1)
df_bui_plot.index = df_bui_plot.index - pd.offsets.YearBegin(1)
df_bui_plot = df_bui_plot.dropna(axis=1, how='all')

# Plot each column
for column in df_bui_plot.columns:
    plt.plot(df_bui_plot.index, df_bui_plot[column], label=column)

# Set major ticks to every 5 years
years = pd.date_range(start='2000', end='2100', freq='5YS')
plt.gca().set_xticks(years)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Add title and labels
#plt.title(f'BUI Time Series for {region}')
plt.title(f'Corrected BUI Time Series for {region}')
plt.xlabel('Year')
plt.ylabel('BUI')
plt.legend()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
           ncol=3, fancybox=True, shadow=True) # Legend below
plt.grid(True)
plt.tight_layout()
#plt.savefig(f'/home/users/clelland/Model/Analysis/CMIP and FWI time series/Ecoregion CSVs/{region}/bui_plot_{region}_2001_2100.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

### DC

#### Process and bias correction

In [ ]:
# DC
df_dc = pd.DataFrame({
    'CEMS': df_cems['DC'],
    'ACCESS_HIST': df_access_hist_fwi['DC'],
    'ACCESS_SSP126': df_access_126_fwi['DC'],
    'ACCESS_SSP245': df_access_245_fwi['DC'],
    'ACCESS_SSP370': df_access_370_fwi['DC'],
    'MRI_HIST': df_mri_hist_fwi['DC'],
    'MRI_SSP126': df_mri_126_fwi['DC'],
    'MRI_SSP245': df_mri_245_fwi['DC'],
    'MRI_SSP370': df_mri_370_fwi['DC'],
})

df_dc.sort_index(inplace=True)
df_dc

In [ ]:
# Bias correction
df_bias = df_dc.copy()

# First, ensure the index is datetime
df_bias.index = pd.to_datetime(df_bias.index)

# Add month as a new column
df_bias['month'] = df_bias.index.month

# Create a dictionary to store results
diff_results = {}

# 1. For 2001–2014 (historical)
hist_period = (df_bias.index >= '2001-01-01') & (df_bias.index <= '2014-12-31')

for model in ['ACCESS_HIST', 'MRI_HIST']:
    diff = df_bias.loc[hist_period, 'CEMS'] - df_bias.loc[hist_period, model]
    monthly_diff = diff.groupby(df_bias.loc[hist_period, 'month']).mean()
    diff_results[model] = monthly_diff

# 2. For 2015–2023 (future scenarios)
future_period = (df_bias.index >= '2015-01-01') & (df_bias.index <= '2023-12-31')

for model in ['ACCESS_SSP126', 'ACCESS_SSP245', 'ACCESS_SSP370',
              'MRI_SSP126', 'MRI_SSP245', 'MRI_SSP370']:
    diff = df_bias.loc[future_period, 'CEMS'] - df_bias.loc[future_period, model]
    monthly_diff = diff.groupby(df_bias.loc[future_period, 'month']).mean()
    diff_results[model] = monthly_diff

# 3. Combine all differences into a dataframe
diff_df = pd.DataFrame(diff_results)

# Optional: sort the dataframe by month
diff_df.index.name = 'month'
diff_df = diff_df.sort_index()

diff_df

In [ ]:
# 1. Define the period you want to correct
hist_correction_period = (df_bias.index >= '2001-01-01') & (df_bias.index <= '2014-12-31')
future_correction_period = (df_bias.index >= '2015-01-01') & (df_bias.index <= '2100-12-31')

# 2. Loop through each model/scenario you want to correct
corrected_data = {}
corrected_data['CEMS'] = df_bias['CEMS']

for model in ['ACCESS_HIST', 'MRI_HIST']:
    # Select the original model data for 2001–2014
    model_data = df_bias.loc[hist_correction_period, model].copy()
    
    # Get the corresponding month for each row
    months = df_bias.loc[hist_correction_period, 'month']
    
    # Subtract the bias correction based on the month
    corrected_values = model_data + months.map(diff_df[model])
    
    # Store in a dictionary
    corrected_data[model] = corrected_values

for model in ['ACCESS_SSP126', 'ACCESS_SSP245', 'ACCESS_SSP370',
              'MRI_SSP126', 'MRI_SSP245', 'MRI_SSP370']:
    # Select the original model data for 2015–2100
    model_data = df_bias.loc[future_correction_period, model].copy()
    
    # Get the corresponding month for each row
    months = df_bias.loc[future_correction_period, 'month']
    
    # Subtract the bias correction based on the month
    corrected_values = model_data + months.map(diff_df[model])
    
    # Store in a dictionary
    corrected_data[model] = corrected_values

# 3. Combine corrected data into a dataframe
corrected_df_dc = pd.DataFrame(corrected_data, index=df_bias.index)

corrected_df_dc

#### Plot

In [ ]:
#df_dc_plot = df_dc.copy() # Original data
df_dc_plot = corrected_df_dc.copy() # Corrected data
df_dc_plot.index = pd.to_datetime(df_dc_plot.index)
#df_dc_plot = df_dc_plot.loc['2001':'2014']
df_dc_plot = df_dc_plot.loc['2001':'2023']
df_dc_plot = df_dc_plot.dropna(axis=1, how='all')

# Set figure size
plt.figure(figsize=(14, 6))

# Plot each column
for column in df_dc_plot.columns:
    plt.plot(df_dc_plot.index, df_dc_plot[column], label=column)

# Set major ticks to every 5 years
#years = pd.date_range(start='2001', end='2015', freq='YS')
years = pd.date_range(start='2001', end='2023', freq='YS')
plt.gca().set_xticks(years)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Add title and labels
#plt.title(f'DC Time Series for {region}')
plt.title(f'Corrected DC Time Series for {region}')
plt.xlabel('Year')
plt.ylabel('DC')
plt.legend()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
           ncol=3, fancybox=True, shadow=True) # Legend below
plt.grid(True)
plt.tight_layout()
#plt.savefig(f'/home/users/clelland/Model/Analysis/CMIP and FWI time series/Ecoregion CSVs/{region}/dc_plot_{region}_2001_2023.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
# Set figure size
plt.figure(figsize=(14, 6))# Set figure size

#df_dc_plot = df_dc.copy() # Original data
df_dc_plot = corrected_df_dc.copy() # Corrected data
df_dc_plot.index = pd.to_datetime(df_dc_plot.index)
df_dc_plot = df_dc_plot.resample('YE').mean()
df_dc_plot.index = df_dc_plot.index + pd.offsets.Day(1)
df_dc_plot.index = df_dc_plot.index - pd.offsets.YearBegin(1)
df_dc_plot = df_dc_plot.dropna(axis=1, how='all')

# Plot each column
for column in df_dc_plot.columns:
    plt.plot(df_dc_plot.index, df_dc_plot[column], label=column)

# Set major ticks to every 5 years
years = pd.date_range(start='2000', end='2100', freq='5YS')
plt.gca().set_xticks(years)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Add title and labels
#plt.title(f'DC Time Series for {region}')
plt.title(f'Corrected DC Time Series for {region}')
plt.xlabel('Year')
plt.ylabel('DC')
plt.legend()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
           ncol=3, fancybox=True, shadow=True) # Legend below
plt.grid(True)
plt.tight_layout()
#plt.savefig(f'/home/users/clelland/Model/Analysis/CMIP and FWI time series/Ecoregion CSVs/{region}/dc_plot_{region}_2001_2100.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

### DMC

#### Process and bias correction

In [ ]:
# DMC
df_dmc = pd.DataFrame({
    'CEMS': df_cems['DMC'],
    'ACCESS_HIST': df_access_hist_fwi['DMC'],
    'ACCESS_SSP126': df_access_126_fwi['DMC'],
    'ACCESS_SSP245': df_access_245_fwi['DMC'],
    'ACCESS_SSP370': df_access_370_fwi['DMC'],
    'MRI_HIST': df_mri_hist_fwi['DMC'],
    'MRI_SSP126': df_mri_126_fwi['DMC'],
    'MRI_SSP245': df_mri_245_fwi['DMC'],
    'MRI_SSP370': df_mri_370_fwi['DMC'],
})

df_dmc.sort_index(inplace=True)
df_dmc

In [ ]:
# Bias correction
df_bias = df_dmc.copy()

# First, ensure the index is datetime
df_bias.index = pd.to_datetime(df_bias.index)

# Add month as a new column
df_bias['month'] = df_bias.index.month

# Create a dictionary to store results
diff_results = {}

# 1. For 2001–2014 (historical)
hist_period = (df_bias.index >= '2001-01-01') & (df_bias.index <= '2014-12-31')

for model in ['ACCESS_HIST', 'MRI_HIST']:
    diff = df_bias.loc[hist_period, 'CEMS'] - df_bias.loc[hist_period, model]
    monthly_diff = diff.groupby(df_bias.loc[hist_period, 'month']).mean()
    diff_results[model] = monthly_diff

# 2. For 2015–2023 (future scenarios)
future_period = (df_bias.index >= '2015-01-01') & (df_bias.index <= '2023-12-31')

for model in ['ACCESS_SSP126', 'ACCESS_SSP245', 'ACCESS_SSP370',
              'MRI_SSP126', 'MRI_SSP245', 'MRI_SSP370']:
    diff = df_bias.loc[future_period, 'CEMS'] - df_bias.loc[future_period, model]
    monthly_diff = diff.groupby(df_bias.loc[future_period, 'month']).mean()
    diff_results[model] = monthly_diff

# 3. Combine all differences into a dataframe
diff_df = pd.DataFrame(diff_results)

# Optional: sort the dataframe by month
diff_df.index.name = 'month'
diff_df = diff_df.sort_index()

diff_df

In [ ]:
# 1. Define the period you want to correct
hist_correction_period = (df_bias.index >= '2001-01-01') & (df_bias.index <= '2014-12-31')
future_correction_period = (df_bias.index >= '2015-01-01') & (df_bias.index <= '2100-12-31')

# 2. Loop through each model/scenario you want to correct
corrected_data = {}
corrected_data['CEMS'] = df_bias['CEMS']

for model in ['ACCESS_HIST', 'MRI_HIST']:
    # Select the original model data for 2001–2014
    model_data = df_bias.loc[hist_correction_period, model].copy()
    
    # Get the corresponding month for each row
    months = df_bias.loc[hist_correction_period, 'month']
    
    # Subtract the bias correction based on the month
    corrected_values = model_data + months.map(diff_df[model])
    
    # Store in a dictionary
    corrected_data[model] = corrected_values

for model in ['ACCESS_SSP126', 'ACCESS_SSP245', 'ACCESS_SSP370',
              'MRI_SSP126', 'MRI_SSP245', 'MRI_SSP370']:
    # Select the original model data for 2015–2100
    model_data = df_bias.loc[future_correction_period, model].copy()
    
    # Get the corresponding month for each row
    months = df_bias.loc[future_correction_period, 'month']
    
    # Subtract the bias correction based on the month
    corrected_values = model_data + months.map(diff_df[model])
    
    # Store in a dictionary
    corrected_data[model] = corrected_values

# 3. Combine corrected data into a dataframe
corrected_df_dmc = pd.DataFrame(corrected_data, index=df_bias.index)

corrected_df_dmc

#### Plot

In [ ]:
#df_dmc_plot = df_dmc.copy() # Original data
df_dmc_plot = corrected_df_dmc.copy() # Corrected data
df_dmc_plot.index = pd.to_datetime(df_dmc_plot.index)
#df_dmc_plot = df_dmc_plot.loc['2001':'2014']
df_dmc_plot = df_dmc_plot.loc['2001':'2023']
df_dmc_plot = df_dmc_plot.dropna(axis=1, how='all')

# Set figure size
plt.figure(figsize=(14, 6))

# Plot each column
for column in df_dmc_plot.columns:
    plt.plot(df_dmc_plot.index, df_dmc_plot[column], label=column)

# Set major ticks to every 5 years
#years = pd.date_range(start='2001', end='2015', freq='YS')
years = pd.date_range(start='2001', end='2023', freq='YS')
plt.gca().set_xticks(years)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Add title and labels
#plt.title(f'DMC Time Series for {region}')
plt.title(f'Corrected DMC Time Series for {region}')
plt.xlabel('Year')
plt.ylabel('DMC')
plt.legend()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
           ncol=3, fancybox=True, shadow=True) # Legend below
plt.grid(True)
plt.tight_layout()
#plt.savefig(f'/home/users/clelland/Model/Analysis/CMIP and FWI time series/Ecoregion CSVs/{region}/dmc_plot_{region}_2001_2023.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
# Set figure size
plt.figure(figsize=(14, 6))

#df_dmc_plot = df_dmc.copy() # Original data
df_dmc_plot = corrected_df_dmc.copy() # Corrected data
df_dmc_plot.index = pd.to_datetime(df_dmc_plot.index)
df_dmc_plot = df_dmc_plot.resample('YE').mean()
df_dmc_plot.index = df_dmc_plot.index + pd.offsets.Day(1)
df_dmc_plot.index = df_dmc_plot.index - pd.offsets.YearBegin(1)
df_dmc_plot = df_dmc_plot.dropna(axis=1, how='all')

# Plot each column
for column in df_dmc_plot.columns:
    plt.plot(df_dmc_plot.index, df_dmc_plot[column], label=column)

# Set major ticks to every 5 years
years = pd.date_range(start='2000', end='2100', freq='5YS')
plt.gca().set_xticks(years)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Add title and labels
#plt.title(f'DMC Time Series for {region}')
plt.title(f'Corrected DMC Time Series for {region}')
plt.xlabel('Year')
plt.ylabel('DMC')
plt.legend()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
           ncol=3, fancybox=True, shadow=True) # Legend below
plt.grid(True)
plt.tight_layout()
#plt.savefig(f'/home/users/clelland/Model/Analysis/CMIP and FWI time series/Ecoregion CSVs/{region}/dmc_plot_{region}_2001_2100.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

### FFMC

#### Process and bias correction

In [ ]:
# FFMC
df_ffmc = pd.DataFrame({
    'CEMS': df_cems['FFMC'],
    'ACCESS_HIST': df_access_hist_fwi['FFMC'],
    'ACCESS_SSP126': df_access_126_fwi['FFMC'],
    'ACCESS_SSP245': df_access_245_fwi['FFMC'],
    'ACCESS_SSP370': df_access_370_fwi['FFMC'],
    'MRI_HIST': df_mri_hist_fwi['FFMC'],
    'MRI_SSP126': df_mri_126_fwi['FFMC'],
    'MRI_SSP245': df_mri_245_fwi['FFMC'],
    'MRI_SSP370': df_mri_370_fwi['FFMC'],
})

df_ffmc.sort_index(inplace=True)
df_ffmc

In [ ]:
# Bias correction
df_bias = df_ffmc.copy()

# First, ensure the index is datetime
df_bias.index = pd.to_datetime(df_bias.index)

# Add month as a new column
df_bias['month'] = df_bias.index.month

# Create a dictionary to store results
diff_results = {}

# 1. For 2001–2014 (historical)
hist_period = (df_bias.index >= '2001-01-01') & (df_bias.index <= '2014-12-31')

for model in ['ACCESS_HIST', 'MRI_HIST']:
    diff = df_bias.loc[hist_period, 'CEMS'] - df_bias.loc[hist_period, model]
    monthly_diff = diff.groupby(df_bias.loc[hist_period, 'month']).mean()
    diff_results[model] = monthly_diff

# 2. For 2015–2023 (future scenarios)
future_period = (df_bias.index >= '2015-01-01') & (df_bias.index <= '2023-12-31')

for model in ['ACCESS_SSP126', 'ACCESS_SSP245', 'ACCESS_SSP370',
              'MRI_SSP126', 'MRI_SSP245', 'MRI_SSP370']:
    diff = df_bias.loc[future_period, 'CEMS'] - df_bias.loc[future_period, model]
    monthly_diff = diff.groupby(df_bias.loc[future_period, 'month']).mean()
    diff_results[model] = monthly_diff

# 3. Combine all differences into a dataframe
diff_df = pd.DataFrame(diff_results)

# Optional: sort the dataframe by month
diff_df.index.name = 'month'
diff_df = diff_df.sort_index()

diff_df

In [ ]:
# 1. Define the period you want to correct
hist_correction_period = (df_bias.index >= '2001-01-01') & (df_bias.index <= '2014-12-31')
future_correction_period = (df_bias.index >= '2015-01-01') & (df_bias.index <= '2100-12-31')

# 2. Loop through each model/scenario you want to correct
corrected_data = {}
corrected_data['CEMS'] = df_bias['CEMS']

for model in ['ACCESS_HIST', 'MRI_HIST']:
    # Select the original model data for 2001–2014
    model_data = df_bias.loc[hist_correction_period, model].copy()
    
    # Get the corresponding month for each row
    months = df_bias.loc[hist_correction_period, 'month']
    
    # Subtract the bias correction based on the month
    corrected_values = model_data + months.map(diff_df[model])
    
    # Store in a dictionary
    corrected_data[model] = corrected_values

for model in ['ACCESS_SSP126', 'ACCESS_SSP245', 'ACCESS_SSP370',
              'MRI_SSP126', 'MRI_SSP245', 'MRI_SSP370']:
    # Select the original model data for 2015–2100
    model_data = df_bias.loc[future_correction_period, model].copy()
    
    # Get the corresponding month for each row
    months = df_bias.loc[future_correction_period, 'month']
    
    # Subtract the bias correction based on the month
    corrected_values = model_data + months.map(diff_df[model])
    
    # Store in a dictionary
    corrected_data[model] = corrected_values

# 3. Combine corrected data into a dataframe
corrected_df_ffmc = pd.DataFrame(corrected_data, index=df_bias.index)

corrected_df_ffmc

#### Plot

In [ ]:
#df_ffmc_plot = df_ffmc.copy() # Original data
df_ffmc_plot = corrected_df_ffmc.copy() # Corrected data
df_ffmc_plot.index = pd.to_datetime(df_ffmc_plot.index)
#df_ffmc_plot = df_ffmc_plot.loc['2001':'2014']
df_ffmc_plot = df_ffmc_plot.loc['2001':'2023']
df_ffmc_plot = df_ffmc_plot.dropna(axis=1, how='all')

# Set figure size
plt.figure(figsize=(14, 6))

# Plot each column
for column in df_ffmc_plot.columns:
    plt.plot(df_ffmc_plot.index, df_ffmc_plot[column], label=column)

# Set major ticks to every 5 years
#years = pd.date_range(start='2001', end='2015', freq='YS')
years = pd.date_range(start='2001', end='2023', freq='YS')
plt.gca().set_xticks(years)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Add title and labels
#plt.title(f'FFMC Time Series for {region}')
plt.title(f'Corrected FFMC Time Series for {region}')
plt.xlabel('Year')
plt.ylabel('FFMC')
plt.legend()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
           ncol=3, fancybox=True, shadow=True) # Legend below
plt.grid(True)
plt.tight_layout()
#plt.savefig(f'/home/users/clelland/Model/Analysis/CMIP and FWI time series/Ecoregion CSVs/{region}/ffmc_plot_{region}_2001_2023.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
# Set figure size
plt.figure(figsize=(14, 6))

#df_ffmc_plot = df_ffmc.copy() # Original data
df_ffmc_plot = corrected_df_ffmc.copy() # Corrected data
df_ffmc_plot.index = pd.to_datetime(df_ffmc_plot.index)
df_ffmc_plot = df_ffmc_plot.resample('YE').mean()
df_ffmc_plot.index = df_ffmc_plot.index + pd.offsets.Day(1)
df_ffmc_plot.index = df_ffmc_plot.index - pd.offsets.YearBegin(1)
df_ffmc_plot = df_ffmc_plot.dropna(axis=1, how='all')

# Plot each column
for column in df_ffmc_plot.columns:
    plt.plot(df_ffmc_plot.index, df_ffmc_plot[column], label=column)

# Set major ticks to every 5 years
years = pd.date_range(start='2000', end='2100', freq='5YS')
plt.gca().set_xticks(years)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Add title and labels
#plt.title(f'FFMC Time Series for {region}')
plt.title(f'Corrected FFMC Time Series for {region}')
plt.xlabel('Year')
plt.ylabel('FFMC')
plt.legend()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
           ncol=3, fancybox=True, shadow=True) # Legend below
plt.grid(True)
plt.tight_layout()
#plt.savefig(f'/home/users/clelland/Model/Analysis/CMIP and FWI time series/Ecoregion CSVs/{region}/ffmc_plot_{region}_2001_2100.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

### FWI

#### Process and bias correction

In [ ]:
# FWI
df_fwi = pd.DataFrame({
    'CEMS': df_cems['FWI'],
    'ACCESS_HIST': df_access_hist_fwi['FWI'],
    'ACCESS_SSP126': df_access_126_fwi['FWI'],
    'ACCESS_SSP245': df_access_245_fwi['FWI'],
    'ACCESS_SSP370': df_access_370_fwi['FWI'],
    'MRI_HIST': df_mri_hist_fwi['FWI'],
    'MRI_SSP126': df_mri_126_fwi['FWI'],
    'MRI_SSP245': df_mri_245_fwi['FWI'],
    'MRI_SSP370': df_mri_370_fwi['FWI'],
})

df_fwi.sort_index(inplace=True)
df_fwi

In [ ]:
# Bias correction
df_bias = df_fwi.copy()

# First, ensure the index is datetime
df_bias.index = pd.to_datetime(df_bias.index)

# Add month as a new column
df_bias['month'] = df_bias.index.month

# Create a dictionary to store results
diff_results = {}

# 1. For 2001–2014 (historical)
hist_period = (df_bias.index >= '2001-01-01') & (df_bias.index <= '2014-12-31')

for model in ['ACCESS_HIST', 'MRI_HIST']:
    diff = df_bias.loc[hist_period, 'CEMS'] - df_bias.loc[hist_period, model]
    monthly_diff = diff.groupby(df_bias.loc[hist_period, 'month']).mean()
    diff_results[model] = monthly_diff

# 2. For 2015–2023 (future scenarios)
future_period = (df_bias.index >= '2015-01-01') & (df_bias.index <= '2023-12-31')

for model in ['ACCESS_SSP126', 'ACCESS_SSP245', 'ACCESS_SSP370',
              'MRI_SSP126', 'MRI_SSP245', 'MRI_SSP370']:
    diff = df_bias.loc[future_period, 'CEMS'] - df_bias.loc[future_period, model]
    monthly_diff = diff.groupby(df_bias.loc[future_period, 'month']).mean()
    diff_results[model] = monthly_diff

# 3. Combine all differences into a dataframe
diff_df = pd.DataFrame(diff_results)

# Optional: sort the dataframe by month
diff_df.index.name = 'month'
diff_df = diff_df.sort_index()

diff_df

In [ ]:
# 1. Define the period you want to correct
hist_correction_period = (df_bias.index >= '2001-01-01') & (df_bias.index <= '2014-12-31')
future_correction_period = (df_bias.index >= '2015-01-01') & (df_bias.index <= '2100-12-31')

# 2. Loop through each model/scenario you want to correct
corrected_data = {}
corrected_data['CEMS'] = df_bias['CEMS']

for model in ['ACCESS_HIST', 'MRI_HIST']:
    # Select the original model data for 2001–2014
    model_data = df_bias.loc[hist_correction_period, model].copy()
    
    # Get the corresponding month for each row
    months = df_bias.loc[hist_correction_period, 'month']
    
    # Subtract the bias correction based on the month
    corrected_values = model_data + months.map(diff_df[model])
    
    # Store in a dictionary
    corrected_data[model] = corrected_values

for model in ['ACCESS_SSP126', 'ACCESS_SSP245', 'ACCESS_SSP370',
              'MRI_SSP126', 'MRI_SSP245', 'MRI_SSP370']:
    # Select the original model data for 2015–2100
    model_data = df_bias.loc[future_correction_period, model].copy()
    
    # Get the corresponding month for each row
    months = df_bias.loc[future_correction_period, 'month']
    
    # Subtract the bias correction based on the month
    corrected_values = model_data + months.map(diff_df[model])
    
    # Store in a dictionary
    corrected_data[model] = corrected_values

# 3. Combine corrected data into a dataframe
corrected_df_fwi = pd.DataFrame(corrected_data, index=df_bias.index)

corrected_df_fwi

#### Plot

In [ ]:
#df_fwi_plot = df_fwi.copy() # Original data
df_fwi_plot = corrected_df_fwi.copy() # Corrected data
df_fwi_plot.index = pd.to_datetime(df_fwi_plot.index)
#df_fwi_plot = df_fwi_plot.loc['2001':'2014']
df_fwi_plot = df_fwi_plot.loc['2001':'2023']
df_fwi_plot = df_fwi_plot.dropna(axis=1, how='all')

# Set figure size
plt.figure(figsize=(14, 6))

# Plot each column
for column in df_fwi_plot.columns:
    plt.plot(df_fwi_plot.index, df_fwi_plot[column], label=column)

# Set major ticks to every 5 years
#years = pd.date_range(start='2001', end='2015', freq='YS')
years = pd.date_range(start='2001', end='2023', freq='YS')
plt.gca().set_xticks(years)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Add title and labels
#plt.title(f'FWI Time Series for {region}')
plt.title(f'Corrected FWI Time Series for {region}')
plt.xlabel('Year')
plt.ylabel('FWI')
plt.legend()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
           ncol=3, fancybox=True, shadow=True) # Legend below
plt.grid(True)
plt.tight_layout()
#plt.savefig(f'/home/users/clelland/Model/Analysis/CMIP and FWI time series/Ecoregion CSVs/{region}/fwi_plot_{region}_2001_2023.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
# Set figure size
plt.figure(figsize=(14, 6))

#df_fwi_plot = df_fwi.copy() # Original data
df_fwi_plot = corrected_df_fwi.copy() # Corrected data
df_fwi_plot.index = pd.to_datetime(df_fwi_plot.index)
df_fwi_plot = df_fwi_plot.resample('YE').mean()
df_fwi_plot.index = df_fwi_plot.index + pd.offsets.Day(1)
df_fwi_plot.index = df_fwi_plot.index - pd.offsets.YearBegin(1)
df_fwi_plot = df_fwi_plot.dropna(axis=1, how='all')

# Plot each column
for column in df_fwi_plot.columns:
    plt.plot(df_fwi_plot.index, df_fwi_plot[column], label=column)

# Set major ticks to every 5 years
years = pd.date_range(start='2000', end='2100', freq='5YS')
plt.gca().set_xticks(years)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Add title and labels
#plt.title(f'FWI Time Series for {region}')
plt.title(f'Corrected FWI Time Series for {region}')
plt.xlabel('Year')
plt.ylabel('FWI')
plt.legend()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
           ncol=3, fancybox=True, shadow=True) # Legend below
plt.grid(True)
plt.tight_layout()
#plt.savefig(f'/home/users/clelland/Model/Analysis/CMIP and FWI time series/Ecoregion CSVs/{region}/fwi_plot_{region}_2001_2100.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

### ISI

#### Process and bias correction

In [ ]:
# ISI
df_isi = pd.DataFrame({
    'CEMS': df_cems['ISI'],
    'ACCESS_HIST': df_access_hist_fwi['ISI'],
    'ACCESS_SSP126': df_access_126_fwi['ISI'],
    'ACCESS_SSP245': df_access_245_fwi['ISI'],
    'ACCESS_SSP370': df_access_370_fwi['ISI'],
    'MRI_HIST': df_mri_hist_fwi['ISI'],
    'MRI_SSP126': df_mri_126_fwi['ISI'],
    'MRI_SSP245': df_mri_245_fwi['ISI'],
    'MRI_SSP370': df_mri_370_fwi['ISI'],
})

df_isi.sort_index(inplace=True)
df_isi

In [ ]:
# Bias correction
df_bias = df_isi.copy()

# First, ensure the index is datetime
df_bias.index = pd.to_datetime(df_bias.index)

# Add month as a new column
df_bias['month'] = df_bias.index.month

# Create a dictionary to store results
diff_results = {}

# 1. For 2001–2014 (historical)
hist_period = (df_bias.index >= '2001-01-01') & (df_bias.index <= '2014-12-31')

for model in ['ACCESS_HIST', 'MRI_HIST']:
    diff = df_bias.loc[hist_period, 'CEMS'] - df_bias.loc[hist_period, model]
    monthly_diff = diff.groupby(df_bias.loc[hist_period, 'month']).mean()
    diff_results[model] = monthly_diff

# 2. For 2015–2023 (future scenarios)
future_period = (df_bias.index >= '2015-01-01') & (df_bias.index <= '2023-12-31')

for model in ['ACCESS_SSP126', 'ACCESS_SSP245', 'ACCESS_SSP370',
              'MRI_SSP126', 'MRI_SSP245', 'MRI_SSP370']:
    diff = df_bias.loc[future_period, 'CEMS'] - df_bias.loc[future_period, model]
    monthly_diff = diff.groupby(df_bias.loc[future_period, 'month']).mean()
    diff_results[model] = monthly_diff

# 3. Combine all differences into a dataframe
diff_df = pd.DataFrame(diff_results)

# Optional: sort the dataframe by month
diff_df.index.name = 'month'
diff_df = diff_df.sort_index()

diff_df

In [ ]:
# 1. Define the period you want to correct
hist_correction_period = (df_bias.index >= '2001-01-01') & (df_bias.index <= '2014-12-31')
future_correction_period = (df_bias.index >= '2015-01-01') & (df_bias.index <= '2100-12-31')

# 2. Loop through each model/scenario you want to correct
corrected_data = {}
corrected_data['CEMS'] = df_bias['CEMS']

for model in ['ACCESS_HIST', 'MRI_HIST']:
    # Select the original model data for 2001–2014
    model_data = df_bias.loc[hist_correction_period, model].copy()
    
    # Get the corresponding month for each row
    months = df_bias.loc[hist_correction_period, 'month']
    
    # Subtract the bias correction based on the month
    corrected_values = model_data + months.map(diff_df[model])
    
    # Store in a dictionary
    corrected_data[model] = corrected_values

for model in ['ACCESS_SSP126', 'ACCESS_SSP245', 'ACCESS_SSP370',
              'MRI_SSP126', 'MRI_SSP245', 'MRI_SSP370']:
    # Select the original model data for 2015–2100
    model_data = df_bias.loc[future_correction_period, model].copy()
    
    # Get the corresponding month for each row
    months = df_bias.loc[future_correction_period, 'month']
    
    # Subtract the bias correction based on the month
    corrected_values = model_data + months.map(diff_df[model])
    
    # Store in a dictionary
    corrected_data[model] = corrected_values

# 3. Combine corrected data into a dataframe
corrected_df_isi = pd.DataFrame(corrected_data, index=df_bias.index)

corrected_df_isi

#### Plot

In [ ]:
#df_isi_plot = df_isi.copy() # Original data
df_isi_plot = corrected_df_isi.copy() # Corrected data
df_isi_plot.index = pd.to_datetime(df_isi_plot.index)
#df_isi_plot = df_isi_plot.loc['2001':'2014']
df_isi_plot = df_isi_plot.loc['2001':'2023']
df_isi_plot = df_isi_plot.dropna(axis=1, how='all')

# Set figure size
plt.figure(figsize=(14, 6))

# Plot each column
for column in df_isi_plot.columns:
    plt.plot(df_isi_plot.index, df_isi_plot[column], label=column)

# Set major ticks to every 5 years
#years = pd.date_range(start='2001', end='2015', freq='YS')
years = pd.date_range(start='2001', end='2023', freq='YS')
plt.gca().set_xticks(years)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Add title and labels
#plt.title(f'ISI Time Series for {region}')
plt.title(f'Corrected ISI Time Series for {region}')
plt.xlabel('Year')
plt.ylabel('ISI')
plt.legend()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
           ncol=3, fancybox=True, shadow=True) # Legend below
plt.grid(True)
plt.tight_layout()
#plt.savefig(f'/home/users/clelland/Model/Analysis/CMIP and FWI time series/Ecoregion CSVs/{region}/isi_plot_{region}_2001_2023.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
# Set figure size
plt.figure(figsize=(14, 6))

#df_isi_plot = df_isi.copy() # Original data
df_isi_plot = corrected_df_isi.copy() # Corrected data
df_isi_plot.index = pd.to_datetime(df_isi_plot.index)
df_isi_plot = df_isi_plot.resample('YE').mean()
df_isi_plot.index = df_isi_plot.index + pd.offsets.Day(1)
df_isi_plot.index = df_isi_plot.index - pd.offsets.YearBegin(1)
df_isi_plot = df_isi_plot.dropna(axis=1, how='all')

# Plot each column
for column in df_isi_plot.columns:
    plt.plot(df_isi_plot.index, df_isi_plot[column], label=column)

# Set major ticks to every 5 years
years = pd.date_range(start='2000', end='2100', freq='5YS')
plt.gca().set_xticks(years)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Add title and labels
#plt.title(f'ISI Time Series for {region}')
plt.title(f'Corrected ISI Time Series for {region}')
plt.xlabel('Year')
plt.ylabel('ISI')
plt.legend()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
           ncol=3, fancybox=True, shadow=True) # Legend below
plt.grid(True)
plt.tight_layout()
#plt.savefig(f'/home/users/clelland/Model/Analysis/CMIP and FWI time series/Ecoregion CSVs/{region}/isi_plot_{region}_2001_2100.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()